<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

You might want to consider the [start](search.ipynb) of this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
VERSION = '2017'

In [4]:
A = use('bhsa', hoist=globals(), version=VERSION)

Using etcbc/bhsa/tf - 2017 r1.4 in /Users/dirk/text-fabric-data
Using etcbc/phono/tf - 2017 r1.1 in /Users/dirk/text-fabric-data
Using etcbc/parallels/tf - 2017 r1.1 in /Users/dirk/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Apps/Bhsa/" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.1.1</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

# Relationships

There are relationship between objects that are about their identities, the way they 
occupy space, and the way they are connected.

Are two objects the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other? Is there an edge from one to another?

Although these relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for pre-computing: most inequality relations are of that kind.
It would require an enormous amount of storage to pre-compute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our relationships here.

Let us first see what relationships we have:

In [5]:
S.relationsLegend()

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

The top of the list are identity and spatial relationships.
We are going to dicuss them. They are documented in
[relationships](https://dans-labs.github.io/text-fabric/Use/Search/#relational-operators)

The bottom of the list are relationships defined by the edge features of your dataset.
We have discussed them in [advanced](searchAdvanced.ipynb).

# Identity and order

## = equal as node

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [6]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
results = A.search(query)
A.table(results, end=3)

  0.89s 48 results


n | p | verse | sentence | clause | phrase | word | verse | sentence | clause | clause | clause
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Jeremiah 44:15">Jeremiah 44:15</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15" sec="Jeremiah 44:15"><span  class="hlup" >Jeremiah 44:15</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Jeremiah 44:15</span></span><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span></span>|<span class="hb"><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Jeremiah 44:15">Jeremiah 44:15</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15" sec="Jeremiah 44:15"><span  class="hlup" >Jeremiah 44:15</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Jeremiah 44:15</span></span><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span></span>|<span class="hb"><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Jeremiah 44:15">Jeremiah 44:15</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15" sec="Jeremiah 44:15"><span  class="hlup" >Jeremiah 44:15</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span  class="hl hlup" >מְקַטְּרֹ֤ות </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Jeremiah 44:15</span></span><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span><span  class="hlup" >כִּֽי־</span><span  class="hl hlup" >מְקַטְּרֹ֤ות </span><span  class="hlup" >נְשֵׁיהֶם֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אלֹהִ֣ים </span><span  class="hlup" >אֲחֵרִ֔ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >עֹמְדֹ֖ות </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעֲנ֣וּ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >יִרְמְיָ֗הוּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אֲנָשִׁ֤ים </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >נָּשִׁ֥ים </span><span  class="hlup" >קָהָ֣ל </span><span  class="hlup" >גָּדֹ֑ול </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עָ֛ם </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽדְעִים֙ </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >יֹּשְׁבִ֥ים </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶֽרֶץ־</span><span  class="hlup" >מִצְרַ֖יִם </span><span  class="hlup" >בְּ</span><span  class="hlup" >פַתְרֹ֥וס </span></span>

In [7]:
A.show(results, start=1, end=1, colorMap={3: 'magenta', 7: 'cyan'})



**verse** *1*



## # unequal as node

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

## < and > canonical order

`n < m` if `n` comes before `m` in the
[canonical ordering](https://dans-labs.github.io/text-fabric/Api/General/#navigating-nodes)
of nodes.

We have seen them in action before.

# Space occupation

We show that the following relationships also work with custom sets, as introduced in the 
[searchAdvanced tutorial](searchAdvanced.ipynb).

We make two custom sets, `common` and `rare`, consisting of nodes whose contained slots have all
common lexemes or some rare lexemes, like we did in
[searchAdvanced](searchAdvanced.ipynb).

In [8]:
COMMON_RANK = 100
RARE_RANK = 500

common = set()
rare = set()

for n in N():
    nTp = F.otype.v(n)
    if nTp == 'lex':
       continue
    if nTp == 'word':
        ranks = [F.rank_lex.v(n)]
    else:
        ranks = [F.rank_lex.v(w) for w in L.d(n, otype='word')]
    maxRank = max(ranks)
    minRank = min(ranks)
    if maxRank < COMMON_RANK:
        common.add(n)
    if maxRank > RARE_RANK:
        rare.add(n)
        
print(f'{len(common):>6} members in set common')
print(f'{len(rare):>6} members in set rare')

669178 members in set common
424746 members in set rare


Now we can do all kinds of searches within the domain of `common` and `rare` things.

We give the names to all the sets and put them in a dictionary.

In [9]:
customSets=dict(
    common=common,
    rare=rare,
)

**Expert remark**

Note that these sets contain both slot nodes and non-slot nodes. 
The code that implements the basic relationship is heavily optimized and contains case distinctions as to whether nodes are slot
or non-slot.
For ordinary node types, it is clear on beforehand whether its nodes are slot or non-slot, but custom sets may contain both.

So our `frequent` and `infrequent` sets are good tests whether the basic relationships are correctly implemented.

## == same slots

Two objects are extensionally equal if they occupy exactly the same slots.

Quite an expensive relation, as you will see: nearly 30 seconds for ca. 3600 results.

This query may take half a minute.

In [10]:
indent(reset=True)
query = '''
v:verse
    s:sentence
v == s
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

    17s 3601 results


n | p | verse | sentence
--- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:16">Genesis 1:16</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:16" sec="Genesis 1:16"><span  class="hlup" >Genesis 1:16</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּ֣עַשׂ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >שְׁנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מְּאֹרֹ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >גְּדֹלִ֑ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מָּאֹ֤ור </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹל֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מֶמְשֶׁ֣לֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּ֔ום </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מָּאֹ֤ור </span><span  class="hlup" >הַ</span><span  class="hlup" >קָּטֹן֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מֶמְשֶׁ֣לֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >כֹּוכָבִֽים׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּ֣עַשׂ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >שְׁנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מְּאֹרֹ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >גְּדֹלִ֑ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מָּאֹ֤ור </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹל֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מֶמְשֶׁ֣לֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּ֔ום </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מָּאֹ֤ור </span><span  class="hlup" >הַ</span><span  class="hlup" >קָּטֹן֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מֶמְשֶׁ֣לֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >כֹּוכָבִֽים׃ </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:1">Genesis 2:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:1" sec="Genesis 2:1"><span  class="hlup" >Genesis 2:1</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְכֻלּ֛וּ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֥יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >צְבָאָֽם׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְכֻלּ֛וּ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֥יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >צְבָאָֽם׃ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:9">Genesis 2:9</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:9" sec="Genesis 2:9"><span  class="hlup" >Genesis 2:9</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּצְמַ֞ח </span><span  class="hlup" >יְהוָ֤ה </span><span  class="hlup" >אֱלֹהִים֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >נֶחְמָ֥ד </span><span  class="hlup" >לְ</span><span  class="hlup" >מַרְאֶ֖ה </span><span  class="hlup" >וְ</span><span  class="hlup" >טֹ֣וב </span><span  class="hlup" >לְ</span><span  class="hlup" >מַאֲכָ֑ל </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֤ץ </span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיִּים֙ </span><span  class="hlup" >בְּ</span><span  class="hlup" >תֹ֣וךְ </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּ֔ן </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֕ץ </span><span  class="hlup" >הַ</span><span  class="hlup" >דַּ֖עַת </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּצְמַ֞ח </span><span  class="hlup" >יְהוָ֤ה </span><span  class="hlup" >אֱלֹהִים֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >נֶחְמָ֥ד </span><span  class="hlup" >לְ</span><span  class="hlup" >מַרְאֶ֖ה </span><span  class="hlup" >וְ</span><span  class="hlup" >טֹ֣וב </span><span  class="hlup" >לְ</span><span  class="hlup" >מַאֲכָ֑ל </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֤ץ </span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיִּים֙ </span><span  class="hlup" >בְּ</span><span  class="hlup" >תֹ֣וךְ </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּ֔ן </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֕ץ </span><span  class="hlup" >הַ</span><span  class="hlup" >דַּ֖עַת </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:5">Genesis 3:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:5" sec="Genesis 3:5"><span  class="hlup" >Genesis 3:5</span></a><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:23">Genesis 3:23</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:23" sec="Genesis 3:23"><span  class="hlup" >Genesis 3:23</span></a><span  class="hlup" >וַֽ</span><span  class="hlup" >יְשַׁלְּחֵ֛הוּ </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >מִ</span><span  class="hlup" >גַּן־</span><span  class="hlup" >עֵ֑דֶן </span><span  class="hlup" >לַֽ</span><span  class="hlup" >עֲבֹד֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >לֻקַּ֖ח </span><span  class="hlup" >מִ</span><span  class="hlup" >שָּֽׁם׃ </span></span>|<span class="hb"><span  class="hlup" >וַֽ</span><span  class="hlup" >יְשַׁלְּחֵ֛הוּ </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >מִ</span><span  class="hlup" >גַּן־</span><span  class="hlup" >עֵ֑דֶן </span><span  class="hlup" >לַֽ</span><span  class="hlup" >עֲבֹד֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >לֻקַּ֖ח </span><span  class="hlup" >מִ</span><span  class="hlup" >שָּֽׁם׃ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 4:24">Genesis 4:24</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:24" sec="Genesis 4:24"><span  class="hlup" >Genesis 4:24</span></a><span  class="hlup" >כִּ֥י </span><span  class="hlup" >שִׁבְעָתַ֖יִם </span><span  class="hlup" >יֻקַּם־</span><span  class="hlup" >קָ֑יִן </span><span  class="hlup" >וְ</span><span  class="hlup" >לֶ֖מֶךְ </span><span  class="hlup" >שִׁבְעִ֥ים </span><span  class="hlup" >וְ</span><span  class="hlup" >שִׁבְעָֽה׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֥י </span><span  class="hlup" >שִׁבְעָתַ֖יִם </span><span  class="hlup" >יֻקַּם־</span><span  class="hlup" >קָ֑יִן </span><span  class="hlup" >וְ</span><span  class="hlup" >לֶ֖מֶךְ </span><span  class="hlup" >שִׁבְעִ֥ים </span><span  class="hlup" >וְ</span><span  class="hlup" >שִׁבְעָֽה׃ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 5:23">Genesis 5:23</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:23" sec="Genesis 5:23"><span  class="hlup" >Genesis 5:23</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְהִ֖י </span><span  class="hlup" >כָּל־</span><span  class="hlup" >יְמֵ֣י </span><span  class="hlup" >חֲנֹ֑וךְ </span><span  class="hlup" >חָמֵ֤שׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >שִׁשִּׁים֙ </span><span  class="hlup" >שָׁנָ֔ה </span><span  class="hlup" >וּ</span><span  class="hlup" >שְׁלֹ֥שׁ </span><span  class="hlup" >מֵאֹ֖ות </span><span  class="hlup" >שָׁנָֽה׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְהִ֖י </span><span  class="hlup" >כָּל־</span><span  class="hlup" >יְמֵ֣י </span><span  class="hlup" >חֲנֹ֑וךְ </span><span  class="hlup" >חָמֵ֤שׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >שִׁשִּׁים֙ </span><span  class="hlup" >שָׁנָ֔ה </span><span  class="hlup" >וּ</span><span  class="hlup" >שְׁלֹ֥שׁ </span><span  class="hlup" >מֵאֹ֖ות </span><span  class="hlup" >שָׁנָֽה׃ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 6:5">Genesis 6:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:5" sec="Genesis 6:5"><span  class="hlup" >Genesis 6:5</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יַּ֣רְא </span><span  class="hlup" >יְהוָ֔ה </span><span  class="hlup" >כִּ֥י </span><span  class="hlup" >רַבָּ֛ה </span><span  class="hlup" >רָעַ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >בָּ</span><span  class="hlup" ></span><span  class="hlup" >אָ֑רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >יֵ֨צֶר֙ </span><span  class="hlup" >מַחְשְׁבֹ֣ת </span><span  class="hlup" >לִבֹּ֔ו </span><span  class="hlup" >רַ֥ק </span><span  class="hlup" >רַ֖ע </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽום׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּ֣רְא </span><span  class="hlup" >יְהוָ֔ה </span><span  class="hlup" >כִּ֥י </span><span  class="hlup" >רַבָּ֛ה </span><span  class="hlup" >רָעַ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָדָ֖ם </span><span  class="hlup" >בָּ</span><span  class="hlup" ></span><span  class="hlup" >אָ֑רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >יֵ֨צֶר֙ </span><span  class="hlup" >מַחְשְׁבֹ֣ת </span><span  class="hlup" >לִבֹּ֔ו </span><span  class="hlup" >רַ֥ק </span><span  class="hlup" >רַ֖ע </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּֽום׃ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 6:8">Genesis 6:8</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:8" sec="Genesis 6:8"><span  class="hlup" >Genesis 6:8</span></a><span  class="hlup" >וְ</span><span  class="hlup" >נֹ֕חַ </span><span  class="hlup" >מָ֥צָא </span><span  class="hlup" >חֵ֖ן </span><span  class="hlup" >בְּ</span><span  class="hlup" >עֵינֵ֥י </span><span  class="hlup" >יְהוָֽה׃ פ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >נֹ֕חַ </span><span  class="hlup" >מָ֥צָא </span><span  class="hlup" >חֵ֖ן </span><span  class="hlup" >בְּ</span><span  class="hlup" >עֵינֵ֥י </span><span  class="hlup" >יְהוָֽה׃ פ </span></span>



**verse** *1*



    17s Done


Now a similar query with the custom sets:

In [11]:
indent(reset=True)
query = '''
v:common otype=verse
    s:common otype=sentence
v == s
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.54s 135 results


n | p | verse | sentence
--- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=6&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 6:10">Exodus 6:10</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=6&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 6:10" sec="Exodus 6:10"><span  class="hlup" >Exodus 6:10</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=6&verse=28&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 6:28">Exodus 6:28</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=6&verse=28&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 6:28" sec="Exodus 6:28"><span  class="hlup" >Exodus 6:28</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְהִ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֨ום </span><span  class="hlup" >דִּבֶּ֧ר </span><span  class="hlup" >יְהוָ֛ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֖ה </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >מִצְרָֽיִם׃ פ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְהִ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֨ום </span><span  class="hlup" >דִּבֶּ֧ר </span><span  class="hlup" >יְהוָ֛ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֖ה </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >מִצְרָֽיִם׃ פ </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 13:1">Exodus 13:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 13:1" sec="Exodus 13:1"><span  class="hlup" >Exodus 13:1</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=14&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 14:1">Exodus 14:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=14&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 14:1" sec="Exodus 14:1"><span  class="hlup" >Exodus 14:1</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהֹוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהֹוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=16&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 16:11">Exodus 16:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=16&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 16:11" sec="Exodus 16:11"><span  class="hlup" >Exodus 16:11</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=20&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 20:1">Exodus 20:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=20&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 20:1" sec="Exodus 20:1"><span  class="hlup" >Exodus 20:1</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֣ר </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֵ֛ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הַ</span><span  class="hlup" >דְּבָרִ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >אֵ֖לֶּה </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ ס </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֣ר </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֵ֛ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >הַ</span><span  class="hlup" >דְּבָרִ֥ים </span><span  class="hlup" >הָ</span><span  class="hlup" >אֵ֖לֶּה </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹֽר׃ ס </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=25&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 25:1">Exodus 25:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=25&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 25:1" sec="Exodus 25:1"><span  class="hlup" >Exodus 25:1</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 30:11">Exodus 30:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:11" sec="Exodus 30:11"><span  class="hlup" >Exodus 30:11</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 30:17">Exodus 30:17</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:17" sec="Exodus 30:17"><span  class="hlup" >Exodus 30:17</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 30:22">Exodus 30:22</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:22" sec="Exodus 30:22"><span  class="hlup" >Exodus 30:22</span></a><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְדַבֵּ֥ר </span><span  class="hlup" >יְהוָ֖ה </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מֹשֶׁ֥ה </span><span  class="hlup" >לֵּ</span><span  class="hlup" >אמֹֽר׃ </span></span>



**verse** *1*



  1.55s Done


As a check we compute manually the maximum rank of the lexemes in the clauses yielded by the query without the custom sets:

In [12]:
rejected = 0
for (verse, sentence) in results:
    maxRank = max(F.rank_lex.v(w) for w in L.d(verse, otype='word'))
    if maxRank >= COMMON_RANK:
        rejected += 1
        
print(f'Rejected {rejected} non-common results, leaving {len(results) - rejected} ones')

Rejected 3466 non-common results, leaving 135 ones


## && overlap

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

We are going to find the sentences that overlap with two verses.

In [13]:
query = '''
chapter
  v1:verse
  v2:verse
  s:sentence

s && v1
s && v2

v1 < v2
'''

This query does not perform well. That's why we call it in a more cautious way,
using the generic Text-Fabric
[`S` - API](https://dans-labs.github.io/text-fabric/Api/General/#search-api).
That will be explained in the [rough](searchRough.ipynb) part of the tutorial.

We fetch only 10 results, and still it may take a minute.

In [14]:
indent(reset=True)
results = S.search(query, limit=10)
info('Done')

    44s Done


In [15]:
A.table(results, end=10)
A.show(results, condensed=False, start=1, end=3)

n | p | chapter | verse | verse | sentence
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:17">Genesis 1:17</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:17</span></span><span  class="hlup" >וַ</span><span  class="hlup" >יִּתֵּ֥ן </span><span  class="hlup" >אֹתָ֛ם </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >בִּ</span><span  class="hlup" >רְקִ֣יעַ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמָ֑יִם </span><span  class="hlup" >לְ</span><span  class="hlup" >הָאִ֖יר </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span class="vn">Genesis 1:18</span><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >מְשֹׁל֙ </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >יֹּ֣ום </span><span  class="hlup" >וּ</span><span  class="hlup" >בַ</span><span  class="hlup" ></span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לֲ</span><span  class="hlup" >הַבְדִּ֔יל </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹ֑שֶׁךְ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּתֵּ֥ן </span><span  class="hlup" >אֹתָ֛ם </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >בִּ</span><span  class="hlup" >רְקִ֣יעַ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמָ֑יִם </span><span  class="hlup" >לְ</span><span  class="hlup" >הָאִ֖יר </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >מְשֹׁל֙ </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >יֹּ֣ום </span><span  class="hlup" >וּ</span><span  class="hlup" >בַ</span><span  class="hlup" ></span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לֲ</span><span  class="hlup" >הַבְדִּ֔יל </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹ֑שֶׁךְ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span class="vn">Genesis 1:29</span>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זֶ֗רַע </span><span  class="hlup" >אֲשֶׁר֙ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּ֥ו </span><span  class="hlup" >פְרִי־</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זָ֑רַע </span>לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span class="vn">Genesis 1:30</span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּו֙ </span><span  class="hlup" >נֶ֣פֶשׁ </span><span  class="hlup" >חַיָּ֔ה </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hlup" >אָכְלָ֑ה </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זֶ֗רַע </span><span  class="hlup" >אֲשֶׁר֙ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּ֥ו </span><span  class="hlup" >פְרִי־</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זָ֑רַע </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּו֙ </span><span  class="hlup" >נֶ֣פֶשׁ </span><span  class="hlup" >חַיָּ֔ה </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hlup" >אָכְלָ֑ה </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:4">Genesis 2:4</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2" sec="Genesis 2">Genesis 2</a></span>|<span class="hb"><span class="vn">Genesis 2:4</span>אֵ֣לֶּה תֹולְדֹ֧ות הַשָּׁמַ֛יִם וְהָאָ֖רֶץ בְּהִבָּֽרְאָ֑ם <span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֗ום </span><span  class="hlup" >עֲשֹׂ֛ות </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >שָׁמָֽיִם׃ </span></span>|<span class="hb"><span class="vn">Genesis 2:7</span><span  class="hlup" >וַ</span><span  class="hlup" >יִּיצֶר֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֗ם </span><span  class="hlup" >עָפָר֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span>וַיִּפַּ֥ח בְּאַפָּ֖יו נִשְׁמַ֣ת חַיִּ֑ים וַֽיְהִ֥י הָֽאָדָ֖ם לְנֶ֥פֶשׁ חַיָּֽה׃ </span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֗ום </span><span  class="hlup" >עֲשֹׂ֛ות </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >שָׁמָֽיִם׃ </span><span  class="hlup" >וַ</span><span  class="hlup" >יִּיצֶר֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֗ם </span><span  class="hlup" >עָפָר֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֲדָמָ֔ה </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 7:2">Genesis 7:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7" sec="Genesis 7">Genesis 7</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:2</span></span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּ֣ל׀ </span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָ֣ה </span><span  class="hlup" >הַ</span><span  class="hlup" >טְּהֹורָ֗ה </span><span  class="hlup" >תִּֽקַּח־</span><span  class="hlup" >לְךָ֛ </span><span  class="hlup" >שִׁבְעָ֥ה </span><span  class="hlup" >שִׁבְעָ֖ה </span><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מִן־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָ֡ה </span><span  class="hlup" >אֲ֠שֶׁר </span><span  class="hlup" >לֹ֣א </span><span  class="hlup" >טְהֹרָ֥ה </span><span  class="hlup" >הִ֛וא </span><span  class="hlup" >שְׁנַ֖יִם </span><span  class="hlup" >אִ֥ישׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּֽו׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:3</span></span><span  class="hlup" >גַּ֣ם </span><span  class="hlup" >מֵ</span><span  class="hlup" >עֹ֧וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֛יִם </span><span  class="hlup" >שִׁבְעָ֥ה </span><span  class="hlup" >שִׁבְעָ֖ה </span><span  class="hlup" >זָכָ֣ר </span><span  class="hlup" >וּ</span><span  class="hlup" >נְקֵבָ֑ה </span><span  class="hlup" >לְ</span><span  class="hlup" >חַיֹּ֥ות </span><span  class="hlup" >זֶ֖רַע </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >מִ</span><span  class="hlup" >כֹּ֣ל׀ </span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָ֣ה </span><span  class="hlup" >הַ</span><span  class="hlup" >טְּהֹורָ֗ה </span><span  class="hlup" >תִּֽקַּח־</span><span  class="hlup" >לְךָ֛ </span><span  class="hlup" >שִׁבְעָ֥ה </span><span  class="hlup" >שִׁבְעָ֖ה </span><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֑ו </span><span  class="hlup" >וּ</span><span  class="hlup" >מִן־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָ֡ה </span><span  class="hlup" >אֲ֠שֶׁר </span><span  class="hlup" >לֹ֣א </span><span  class="hlup" >טְהֹרָ֥ה </span><span  class="hlup" >הִ֛וא </span><span  class="hlup" >שְׁנַ֖יִם </span><span  class="hlup" >אִ֥ישׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּֽו׃ </span><span  class="hlup" >גַּ֣ם </span><span  class="hlup" >מֵ</span><span  class="hlup" >עֹ֧וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֛יִם </span><span  class="hlup" >שִׁבְעָ֥ה </span><span  class="hlup" >שִׁבְעָ֖ה </span><span  class="hlup" >זָכָ֣ר </span><span  class="hlup" >וּ</span><span  class="hlup" >נְקֵבָ֑ה </span><span  class="hlup" >לְ</span><span  class="hlup" >חַיֹּ֥ות </span><span  class="hlup" >זֶ֖רַע </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 7:8">Genesis 7:8</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7" sec="Genesis 7">Genesis 7</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:8</span></span><span  class="hlup" >מִן־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָה֙ </span><span  class="hlup" >הַ</span><span  class="hlup" >טְּהֹורָ֔ה </span><span  class="hlup" >וּ</span><span  class="hlup" >מִן־</span><span  class="hlup" >הַ֨</span><span  class="hlup" >בְּהֵמָ֔ה </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >אֵינֶ֖נָּה </span><span  class="hlup" >טְהֹרָ֑ה </span><span  class="hlup" >וּ</span><span  class="hlup" >מִ֨ן־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֹ֔וף </span><span  class="hlup" >וְ</span><span  class="hlup" >כֹ֥ל </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >רֹמֵ֖שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אֲדָמָֽה׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:9</span></span><span  class="hlup" >שְׁנַ֨יִם </span><span  class="hlup" >שְׁנַ֜יִם </span><span  class="hlup" >בָּ֧אוּ </span><span  class="hlup" >אֶל־</span><span  class="hlup" >נֹ֛חַ </span><span  class="hlup" >אֶל־</span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָ֖ה </span><span  class="hlup" >זָכָ֣ר </span><span  class="hlup" >וּ</span><span  class="hlup" >נְקֵבָ֑ה </span><span  class="hlup" >כַּֽ</span><span  class="hlup" >אֲשֶׁ֛ר </span><span  class="hlup" >צִוָּ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >נֹֽחַ׃ </span></span>|<span class="hb"><span  class="hlup" >מִן־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָה֙ </span><span  class="hlup" >הַ</span><span  class="hlup" >טְּהֹורָ֔ה </span><span  class="hlup" >וּ</span><span  class="hlup" >מִן־</span><span  class="hlup" >הַ֨</span><span  class="hlup" >בְּהֵמָ֔ה </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >אֵינֶ֖נָּה </span><span  class="hlup" >טְהֹרָ֑ה </span><span  class="hlup" >וּ</span><span  class="hlup" >מִ֨ן־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֹ֔וף </span><span  class="hlup" >וְ</span><span  class="hlup" >כֹ֥ל </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >רֹמֵ֖שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אֲדָמָֽה׃ </span><span  class="hlup" >שְׁנַ֨יִם </span><span  class="hlup" >שְׁנַ֜יִם </span><span  class="hlup" >בָּ֧אוּ </span><span  class="hlup" >אֶל־</span><span  class="hlup" >נֹ֛חַ </span><span  class="hlup" >אֶל־</span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָ֖ה </span><span  class="hlup" >זָכָ֣ר </span><span  class="hlup" >וּ</span><span  class="hlup" >נְקֵבָ֑ה </span><span  class="hlup" >כַּֽ</span><span  class="hlup" >אֲשֶׁ֛ר </span><span  class="hlup" >צִוָּ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >נֹֽחַ׃ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 7:13">Genesis 7:13</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7" sec="Genesis 7">Genesis 7</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:13</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >עֶ֨צֶם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּ֤ום </span><span  class="hlup" >הַ</span><span  class="hlup" >זֶּה֙ </span><span  class="hlup" >בָּ֣א </span><span  class="hlup" >נֹ֔חַ </span><span  class="hlup" >וְ</span><span  class="hlup" >שֵׁם־</span><span  class="hlup" >וְ</span><span  class="hlup" >חָ֥ם </span><span  class="hlup" >וָ</span><span  class="hlup" >יֶ֖פֶת </span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >נֹ֑חַ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֣שֶׁת </span><span  class="hlup" >נֹ֗חַ </span><span  class="hlup" >וּ</span><span  class="hlup" >שְׁלֹ֧שֶׁת </span><span  class="hlup" >נְשֵֽׁי־</span><span  class="hlup" >בָנָ֛יו </span><span  class="hlup" >אִתָּ֖ם </span><span  class="hlup" >אֶל־</span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָֽה׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 7:14</span></span><span  class="hlup" >הֵ֜מָּה </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיָּ֣ה </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינָ֗הּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָה֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינָ֔הּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >רֶ֛מֶשׂ </span><span  class="hlup" >הָ</span><span  class="hlup" >רֹמֵ֥שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֹ֣וף </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >כֹּ֖ל </span><span  class="hlup" >צִפֹּ֥ור </span><span  class="hlup" >כָּל־</span><span  class="hlup" >כָּנָֽף׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >עֶ֨צֶם </span><span  class="hlup" >הַ</span><span  class="hlup" >יֹּ֤ום </span><span  class="hlup" >הַ</span><span  class="hlup" >זֶּה֙ </span><span  class="hlup" >בָּ֣א </span><span  class="hlup" >נֹ֔חַ </span><span  class="hlup" >וְ</span><span  class="hlup" >שֵׁם־</span><span  class="hlup" >וְ</span><span  class="hlup" >חָ֥ם </span><span  class="hlup" >וָ</span><span  class="hlup" >יֶ֖פֶת </span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >נֹ֑חַ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֣שֶׁת </span><span  class="hlup" >נֹ֗חַ </span><span  class="hlup" >וּ</span><span  class="hlup" >שְׁלֹ֧שֶׁת </span><span  class="hlup" >נְשֵֽׁי־</span><span  class="hlup" >בָנָ֛יו </span><span  class="hlup" >אִתָּ֖ם </span><span  class="hlup" >אֶל־</span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָֽה׃ </span><span  class="hlup" >הֵ֜מָּה </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיָּ֣ה </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינָ֗הּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הַ</span><span  class="hlup" >בְּהֵמָה֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינָ֔הּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >רֶ֛מֶשׂ </span><span  class="hlup" >הָ</span><span  class="hlup" >רֹמֵ֥שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֹ֣וף </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >כֹּ֖ל </span><span  class="hlup" >צִפֹּ֥ור </span><span  class="hlup" >כָּל־</span><span  class="hlup" >כָּנָֽף׃ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=9&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 9:9">Genesis 9:9</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=9&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 9" sec="Genesis 9">Genesis 9</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 9:9</span></span><span  class="hlup" >וַ</span><span  class="hlup" >אֲנִ֕י </span><span  class="hlup" >הִנְנִ֥י </span><span  class="hlup" >מֵקִ֛ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >בְּרִיתִ֖י </span><span  class="hlup" >אִתְּכֶ֑ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >זַרְעֲכֶ֖ם </span><span  class="hlup" >אַֽחֲרֵיכֶֽם׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 9:10</span></span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֨ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >נֶ֤פֶשׁ </span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיָּה֙ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >אִתְּכֶ֔ם </span><span  class="hlup" >בָּ</span><span  class="hlup" ></span><span  class="hlup" >עֹ֧וף </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >בְּהֵמָ֛ה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >בְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >אִתְּכֶ֑ם </span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּל֙ </span><span  class="hlup" >יֹצְאֵ֣י </span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָ֔ה </span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֖ל </span><span  class="hlup" >חַיַּ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >אֲנִ֕י </span><span  class="hlup" >הִנְנִ֥י </span><span  class="hlup" >מֵקִ֛ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >בְּרִיתִ֖י </span><span  class="hlup" >אִתְּכֶ֑ם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >זַרְעֲכֶ֖ם </span><span  class="hlup" >אַֽחֲרֵיכֶֽם׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֨ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >נֶ֤פֶשׁ </span><span  class="hlup" >הַֽ</span><span  class="hlup" >חַיָּה֙ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >אִתְּכֶ֔ם </span><span  class="hlup" >בָּ</span><span  class="hlup" ></span><span  class="hlup" >עֹ֧וף </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >בְּהֵמָ֛ה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >בְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֖רֶץ </span><span  class="hlup" >אִתְּכֶ֑ם </span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּל֙ </span><span  class="hlup" >יֹצְאֵ֣י </span><span  class="hlup" >הַ</span><span  class="hlup" >תֵּבָ֔ה </span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֖ל </span><span  class="hlup" >חַיַּ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:11">Genesis 10:11</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10" sec="Genesis 10">Genesis 10</a></span>|<span class="hb"><span class="vn">Genesis 10:11</span>מִן־הָאָ֥רֶץ הַהִ֖וא יָצָ֣א אַשּׁ֑וּר <span  class="hlup" >וַ</span><span  class="hlup" >יִּ֨בֶן֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >נִ֣ינְוֵ֔ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >רְחֹבֹ֥ת עִ֖יר </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּֽלַח׃ </span></span>|<span class="hb"><span class="vn">Genesis 10:12</span><span  class="hlup" >וְֽ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >רֶ֔סֶן </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >נִֽינְוֵ֖ה </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >כָּ֑לַח </span>הִ֖וא הָעִ֥יר הַגְּדֹלָֽה׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּ֨בֶן֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >נִ֣ינְוֵ֔ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >רְחֹבֹ֥ת עִ֖יר </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּֽלַח׃ </span><span  class="hlup" >וְֽ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >רֶ֔סֶן </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >נִֽינְוֵ֖ה </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >כָּ֑לַח </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:13">Genesis 10:13</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10" sec="Genesis 10">Genesis 10</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 10:13</span></span><span  class="hlup" >וּ</span><span  class="hlup" >מִצְרַ֡יִם </span><span  class="hlup" >יָלַ֞ד </span><span  class="hlup" >אֶת־</span><span  class="hlup" >לוּדִ֧ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >עֲנָמִ֛ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >לְהָבִ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >נַפְתֻּחִֽים׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 10:14</span></span><span  class="hlup" >וְֽ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >פַּתְרֻסִ֞ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כַּסְלֻחִ֗ים </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >יָצְא֥וּ </span><span  class="hlup" >מִ</span><span  class="hlup" >שָּׁ֛ם </span><span  class="hlup" >פְּלִשְׁתִּ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כַּפְתֹּרִֽים׃ ס </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >מִצְרַ֡יִם </span><span  class="hlup" >יָלַ֞ד </span><span  class="hlup" >אֶת־</span><span  class="hlup" >לוּדִ֧ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >עֲנָמִ֛ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >לְהָבִ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >נַפְתֻּחִֽים׃ </span><span  class="hlup" >וְֽ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >פַּתְרֻסִ֞ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כַּסְלֻחִ֗ים </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >יָצְא֥וּ </span><span  class="hlup" >מִ</span><span  class="hlup" >שָּׁ֛ם </span><span  class="hlup" >פְּלִשְׁתִּ֖ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כַּפְתֹּרִֽים׃ ס </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:15">Genesis 10:15</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10" sec="Genesis 10">Genesis 10</a></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 10:15</span></span><span  class="hlup" >וּ</span><span  class="hlup" >כְנַ֗עַן </span><span  class="hlup" >יָלַ֛ד </span><span  class="hlup" >אֶת־</span><span  class="hlup" >צִידֹ֥ן </span><span  class="hlup" >בְּכֹרֹ֖ו </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >חֵֽת׃ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 10:16</span></span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >יְבוּסִי֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֱמֹרִ֔י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >גִּרְגָּשִֽׁי׃ </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >כְנַ֗עַן </span><span  class="hlup" >יָלַ֛ד </span><span  class="hlup" >אֶת־</span><span  class="hlup" >צִידֹ֥ן </span><span  class="hlup" >בְּכֹרֹ֖ו </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >חֵֽת׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >יְבוּסִי֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ֣</span><span  class="hlup" >אֱמֹרִ֔י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֖ת </span><span  class="hlup" >הַ</span><span  class="hlup" >גִּרְגָּשִֽׁי׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַֽ</span><span  class="hlup" >חִוִּ֥י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַֽ</span><span  class="hlup" >עַרְקִ֖י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >סִּינִֽי׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אַרְוָדִ֥י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >צְּמָרִ֖י </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַֽ</span><span  class="hlup" >חֲמָתִ֑י </span></span>



**chapter** *1*





**chapter** *2*





**chapter** *3*



Now with custom sets:

In [21]:
query = '''
chapter
  v1:verse
  v2:verse
  s:common otype=sentence

s && v1
s && v2

v1 < v2
'''

This is even worse in performance. We are content with just one result.

In [22]:
indent(reset=True)
resultsCommon = S.search(query, sets=customSets, limit=1)
info('Done')

    55s Done


In [23]:
A.table(resultsCommon)

n | p | chapter | verse | verse | sentence
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=27&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Numbers 27:8">Numbers 27:8</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=27&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 27" sec="Numbers 27">Numbers 27</a></span>|<span class="hb"><span class="vn">Numbers 27:8</span>וְאֶל־בְּנֵ֥י יִשְׂרָאֵ֖ל תְּדַבֵּ֣ר לֵאמֹ֑ר <span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >כִּֽי־</span><span  class="hlup" >יָמ֗וּת </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵן֙ </span><span  class="hlup" >אֵ֣ין </span><span  class="hlup" >לֹ֔ו </span>וְהַֽעֲבַרְתֶּ֥ם אֶת־נַחֲלָתֹ֖ו לְבִתֹּֽו׃ </span>|<span class="hb"><span class="vn">Numbers 27:9</span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >אֵ֥ין </span><span  class="hlup" >לֹ֖ו </span><span  class="hlup" >בַּ֑ת </span>וּנְתַתֶּ֥ם אֶת־נַחֲלָתֹ֖ו לְאֶחָיו׃ </span>|<span class="hb"><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >כִּֽי־</span><span  class="hlup" >יָמ֗וּת </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵן֙ </span><span  class="hlup" >אֵ֣ין </span><span  class="hlup" >לֹ֔ו </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >אֵ֥ין </span><span  class="hlup" >לֹ֖ו </span><span  class="hlup" >בַּ֑ת </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >אֵ֥ין </span><span  class="hlup" >לֹ֖ו </span><span  class="hlup" >אַחִ֑ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >אֵ֣ין </span><span  class="hlup" >אַחִים֮ </span><span  class="hlup" >לְ</span><span  class="hlup" >אָבִיו֒ </span></span>

The following query is much faster and obtains all results!

In [16]:
query = '''
sentence
  =: w1:word
  w2:word
  :=

v1:verse
  wv1:word

v2:verse
  wv2:word

w1 = wv1
w2 = wv2
v1 < v2
'''

In [17]:
indent(reset=True)
results = A.search(query)
info('Done')

  2.01s 892 results
  2.01s Done


In [19]:
A.table(results, end=3)

n | p | sentence | word | word | verse | word | verse | word
--- | --- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:17">Genesis 1:17</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:17" sec="Genesis 1:17"><span  class="hl hlup" >וַ</span><span  class="hlup" >יִּתֵּ֥ן </span><span  class="hlup" >אֹתָ֛ם </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >בִּ</span><span  class="hlup" >רְקִ֣יעַ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמָ֑יִם </span><span  class="hlup" >לְ</span><span  class="hlup" >הָאִ֖יר </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >מְשֹׁל֙ </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >יֹּ֣ום </span><span  class="hlup" >וּ</span><span  class="hlup" >בַ</span><span  class="hlup" ></span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לֲ</span><span  class="hlup" >הַבְדִּ֔יל </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hl hlup" >חֹ֑שֶׁךְ </span></a></span>|<span class="hb"><span  class="hl hlup" >וַ</span></span>|<span class="hb"><span  class="hl hlup" >חֹ֑שֶׁךְ </span></span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:17</span></span><span  class="hl hlup" >וַ</span><span  class="hlup" >יִּתֵּ֥ן </span><span  class="hlup" >אֹתָ֛ם </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >בִּ</span><span  class="hlup" >רְקִ֣יעַ </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמָ֑יִם </span><span  class="hlup" >לְ</span><span  class="hlup" >הָאִ֖יר </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >וַ</span></span>|<span class="hb"><span class="vn">Genesis 1:18</span><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >מְשֹׁל֙ </span><span  class="hlup" >בַּ</span><span  class="hlup" ></span><span  class="hlup" >יֹּ֣ום </span><span  class="hlup" >וּ</span><span  class="hlup" >בַ</span><span  class="hlup" ></span><span  class="hlup" >לַּ֔יְלָה </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לֲ</span><span  class="hlup" >הַבְדִּ֔יל </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hl hlup" >חֹ֑שֶׁךְ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hl hlup" >חֹ֑שֶׁךְ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29"><span  class="hl hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זֶ֗רַע </span><span  class="hlup" >אֲשֶׁר֙ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּ֥ו </span><span  class="hlup" >פְרִי־</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זָ֑רַע </span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּו֙ </span><span  class="hlup" >נֶ֣פֶשׁ </span><span  class="hlup" >חַיָּ֔ה </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hl hlup" >אָכְלָ֑ה </span></a></span>|<span class="hb"><span  class="hl hlup" >הִנֵּה֩ </span></span>|<span class="hb"><span  class="hl hlup" >אָכְלָ֑ה </span></span>|<span class="hb"><span class="vn">Genesis 1:29</span>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hl hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זֶ֗רַע </span><span  class="hlup" >אֲשֶׁר֙ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּ֥ו </span><span  class="hlup" >פְרִי־</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >זֹרֵ֣עַ </span><span  class="hlup" >זָ֑רַע </span>לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hl hlup" >הִנֵּה֩ </span></span>|<span class="hb"><span class="vn">Genesis 1:30</span><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >בֹּו֙ </span><span  class="hlup" >נֶ֣פֶשׁ </span><span  class="hlup" >חַיָּ֔ה </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hl hlup" >אָכְלָ֑ה </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hl hlup" >אָכְלָ֑ה </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:4">Genesis 2:4</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:4" sec="Genesis 2:4"><span  class="hl hlup" >בְּ</span><span  class="hlup" >יֹ֗ום </span><span  class="hlup" >עֲשֹׂ֛ות </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >שָׁמָֽיִם׃ </span><span  class="hlup" >וַ</span><span  class="hlup" >יִּיצֶר֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֗ם </span><span  class="hlup" >עָפָר֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hl hlup" >אֲדָמָ֔ה </span></a></span>|<span class="hb"><span  class="hl hlup" >בְּ</span></span>|<span class="hb"><span  class="hl hlup" >אֲדָמָ֔ה </span></span>|<span class="hb"><span class="vn">Genesis 2:4</span>אֵ֣לֶּה תֹולְדֹ֧ות הַשָּׁמַ֛יִם וְהָאָ֖רֶץ בְּהִבָּֽרְאָ֑ם <span  class="hl hlup" >בְּ</span><span  class="hlup" >יֹ֗ום </span><span  class="hlup" >עֲשֹׂ֛ות </span><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span><span  class="hlup" >אֶ֥רֶץ </span><span  class="hlup" >וְ</span><span  class="hlup" >שָׁמָֽיִם׃ </span></span>|<span class="hb"><span  class="hl hlup" >בְּ</span></span>|<span class="hb"><span class="vn">Genesis 2:7</span><span  class="hlup" >וַ</span><span  class="hlup" >יִּיצֶר֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָֽ</span><span  class="hlup" >אָדָ֗ם </span><span  class="hlup" >עָפָר֙ </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ֣</span><span  class="hl hlup" >אֲדָמָ֔ה </span>וַיִּפַּ֥ח בְּאַפָּ֖יו נִשְׁמַ֣ת חַיִּ֑ים וַֽיְהִ֥י הָֽאָדָ֖ם לְנֶ֥פֶשׁ חַיָּֽה׃ </span>|<span class="hb"><span  class="hl hlup" >אֲדָמָ֔ה </span></span>

In [20]:
A.show(results, end=1)



**sentence** *1*



## ## not the same slots

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

We look for sentences that start with a sentence atom that is not co-extensive with its sentence.

In [24]:
indent(reset=True)
query = '''
verse
  s:sentence
    =: sa:sentence_atom
  s ## sa
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

  0.50s 454 results


n | p | verse | sentence | sentence_atom
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:3">Genesis 3:3</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:3" sec="Genesis 3:3">Genesis 3:3</a><span  class="hlup" >וּ</span><span  class="hlup" >מִ</span><span  class="hlup" >פְּרִ֣י </span><span  class="hlup" >הָ</span><span  class="hlup" >עֵץ֮ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >בְּ</span><span  class="hlup" >תֹוךְ־</span><span  class="hlup" >הַ</span><span  class="hlup" >גָּן֒ </span>אָמַ֣ר אֱלֹהִ֗ים <span  class="hlup" >לֹ֤א </span><span  class="hlup" >תֹֽאכְלוּ֙ </span><span  class="hlup" >מִמֶּ֔נּוּ </span>וְלֹ֥א תִגְּע֖וּ בֹּ֑ו פֶּן־תְּמֻתֽוּן׃ </span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >מִ</span><span  class="hlup" >פְּרִ֣י </span><span  class="hlup" >הָ</span><span  class="hlup" >עֵץ֮ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >בְּ</span><span  class="hlup" >תֹוךְ־</span><span  class="hlup" >הַ</span><span  class="hlup" >גָּן֒ </span><span  class="hlup" >לֹ֤א </span><span  class="hlup" >תֹֽאכְלוּ֙ </span><span  class="hlup" >מִמֶּ֔נּוּ </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >מִ</span><span  class="hlup" >פְּרִ֣י </span><span  class="hlup" >הָ</span><span  class="hlup" >עֵץ֮ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >בְּ</span><span  class="hlup" >תֹוךְ־</span><span  class="hlup" >הַ</span><span  class="hlup" >גָּן֒ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 13:1">Genesis 13:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 13:1" sec="Genesis 13:1">Genesis 13:1</a><span  class="hlup" >וַ</span><span  class="hlup" >יַּעַל֩ </span><span  class="hlup" >אַבְרָ֨ם </span><span  class="hlup" >מִ</span><span  class="hlup" >מִּצְרַ֜יִם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >לֹ֛ו </span>וְלֹ֥וט עִמֹּ֖ו <span  class="hlup" >הַ</span><span  class="hlup" >נֶּֽגְבָּה׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעַל֩ </span><span  class="hlup" >אַבְרָ֨ם </span><span  class="hlup" >מִ</span><span  class="hlup" >מִּצְרַ֜יִם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >לֹ֛ו </span><span  class="hlup" >הַ</span><span  class="hlup" >נֶּֽגְבָּה׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּעַל֩ </span><span  class="hlup" >אַבְרָ֨ם </span><span  class="hlup" >מִ</span><span  class="hlup" >מִּצְרַ֜יִם </span><span  class="hlup" >ה֠וּא </span><span  class="hlup" >וְ</span><span  class="hlup" >אִשְׁתֹּ֧ו </span><span  class="hlup" >וְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >לֹ֛ו </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 13:9">Genesis 13:9</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=13&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 13:9" sec="Genesis 13:9">Genesis 13:9</a>הֲלֹ֤א כָל־הָאָ֨רֶץ֙ לְפָנֶ֔יךָ <span  class="hlup" >הִפָּ֥רֶד </span><span  class="hlup" >נָ֖א </span><span  class="hlup" >מֵ</span><span  class="hlup" >עָלָ֑י </span><span  class="hlup" >אִם־</span><span  class="hlup" >הַ</span><span  class="hlup" >שְּׂמֹ֣אל </span>וְאֵימִ֔נָה <span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >הַ</span><span  class="hlup" >יָּמִ֖ין </span>וְאַשְׂמְאִֽילָה׃ </span>|<span class="hb"><span  class="hlup" >הִפָּ֥רֶד </span><span  class="hlup" >נָ֖א </span><span  class="hlup" >מֵ</span><span  class="hlup" >עָלָ֑י </span><span  class="hlup" >אִם־</span><span  class="hlup" >הַ</span><span  class="hlup" >שְּׂמֹ֣אל </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >הַ</span><span  class="hlup" >יָּמִ֖ין </span></span>|<span class="hb"><span  class="hlup" >הִפָּ֥רֶד </span><span  class="hlup" >נָ֖א </span><span  class="hlup" >מֵ</span><span  class="hlup" >עָלָ֑י </span><span  class="hlup" >אִם־</span><span  class="hlup" >הַ</span><span  class="hlup" >שְּׂמֹ֣אל </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=21&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 21:14">Genesis 21:14</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=21&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 21:14" sec="Genesis 21:14">Genesis 21:14</a>וַיַּשְׁכֵּ֣ם אַבְרָהָ֣ם׀ בַּבֹּ֡קֶר <span  class="hlup" >וַ</span><span  class="hlup" >יִּֽקַּֽח־</span><span  class="hlup" >לֶחֶם֩ </span><span  class="hlup" >וְ</span><span  class="hlup" >חֵ֨מַת </span><span  class="hlup" >מַ֜יִם </span>וַיִּתֵּ֣ן אֶל־הָ֠גָר שָׂ֧ם עַל־שִׁכְמָ֛הּ <span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >יֶּ֖לֶד </span>וַֽיְשַׁלְּחֶ֑הָ וַתֵּ֣לֶךְ וַתֵּ֔תַע בְּמִדְבַּ֖ר בְּאֵ֥ר שָֽׁבַע׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽקַּֽח־</span><span  class="hlup" >לֶחֶם֩ </span><span  class="hlup" >וְ</span><span  class="hlup" >חֵ֨מַת </span><span  class="hlup" >מַ֜יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >יֶּ֖לֶד </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּֽקַּֽח־</span><span  class="hlup" >לֶחֶם֩ </span><span  class="hlup" >וְ</span><span  class="hlup" >חֵ֨מַת </span><span  class="hlup" >מַ֜יִם </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 22:24">Genesis 22:24</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:24" sec="Genesis 22:24">Genesis 22:24</a><span  class="hlup" >וּ</span><span  class="hlup" >פִֽילַגְשֹׁ֖ו </span>וּשְׁמָ֣הּ רְאוּמָ֑ה <span  class="hlup" >וַ</span><span  class="hlup" >תֵּ֤לֶד </span><span  class="hlup" >גַּם־</span><span  class="hlup" >הִוא֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >טֶ֣בַח </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >גַּ֔חַם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >תַּ֖חַשׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >מַעֲכָֽה׃ ס </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >פִֽילַגְשֹׁ֖ו </span><span  class="hlup" >וַ</span><span  class="hlup" >תֵּ֤לֶד </span><span  class="hlup" >גַּם־</span><span  class="hlup" >הִוא֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >טֶ֣בַח </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >גַּ֔חַם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >תַּ֖חַשׁ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >מַעֲכָֽה׃ ס </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >פִֽילַגְשֹׁ֖ו </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=23&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 23:2">Genesis 23:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=23&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 23:2" sec="Genesis 23:2">Genesis 23:2</a><span  class="hlup" >וַ</span><span  class="hlup" >תָּ֣מָת </span><span  class="hlup" >שָׂרָ֗ה </span><span  class="hlup" >בְּ</span><span  class="hlup" >קִרְיַ֥ת אַרְבַּ֛ע </span>הִ֥וא חֶבְרֹ֖ון <span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֣רֶץ </span><span  class="hlup" >כְּנָ֑עַן </span>וַיָּבֹא֙ אַבְרָהָ֔ם לִסְפֹּ֥ד לְשָׂרָ֖ה וְלִבְכֹּתָֽהּ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תָּ֣מָת </span><span  class="hlup" >שָׂרָ֗ה </span><span  class="hlup" >בְּ</span><span  class="hlup" >קִרְיַ֥ת אַרְבַּ֛ע </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֣רֶץ </span><span  class="hlup" >כְּנָ֑עַן </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תָּ֣מָת </span><span  class="hlup" >שָׂרָ֗ה </span><span  class="hlup" >בְּ</span><span  class="hlup" >קִרְיַ֥ת אַרְבַּ֛ע </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=23&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 23:19">Genesis 23:19</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=23&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 23:19" sec="Genesis 23:19">Genesis 23:19</a><span  class="hlup" >וְ</span><span  class="hlup" >אַחֲרֵי־</span><span  class="hlup" >כֵן֩ </span><span  class="hlup" >קָבַ֨ר </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >שָׂרָ֣ה </span><span  class="hlup" >אִשְׁתֹּ֗ו </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מְעָרַ֞ת </span><span  class="hlup" >שְׂדֵ֧ה </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּכְפֵּלָ֛ה </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >מַמְרֵ֖א </span>הִ֣וא חֶבְרֹ֑ון <span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֖רֶץ </span><span  class="hlup" >כְּנָֽעַן׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אַחֲרֵי־</span><span  class="hlup" >כֵן֩ </span><span  class="hlup" >קָבַ֨ר </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >שָׂרָ֣ה </span><span  class="hlup" >אִשְׁתֹּ֗ו </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מְעָרַ֞ת </span><span  class="hlup" >שְׂדֵ֧ה </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּכְפֵּלָ֛ה </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >מַמְרֵ֖א </span><span  class="hlup" >בְּ</span><span  class="hlup" >אֶ֖רֶץ </span><span  class="hlup" >כְּנָֽעַן׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אַחֲרֵי־</span><span  class="hlup" >כֵן֩ </span><span  class="hlup" >קָבַ֨ר </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >שָׂרָ֣ה </span><span  class="hlup" >אִשְׁתֹּ֗ו </span><span  class="hlup" >אֶל־</span><span  class="hlup" >מְעָרַ֞ת </span><span  class="hlup" >שְׂדֵ֧ה </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּכְפֵּלָ֛ה </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >מַמְרֵ֖א </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 24:7">Genesis 24:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:7" sec="Genesis 24:7">Genesis 24:7</a><span  class="hlup" >יְהוָ֣ה׀ </span><span  class="hlup" >אֱלֹהֵ֣י </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֗יִם </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >לְקָחַ֜נִי </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >אָבִי֮ </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵ</span><span  class="hlup" >אֶ֣רֶץ </span><span  class="hlup" >מֹֽולַדְתִּי֒ </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >דִּבֶּר־</span><span  class="hlup" >לִ֜י </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֤ר </span><span  class="hlup" >נִֽשְׁבַּֽע־</span><span  class="hlup" >לִי֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹ֔ר </span>לְזַ֨רְעֲךָ֔ אֶתֵּ֖ן אֶת־הָאָ֣רֶץ הַזֹּ֑את <span  class="hlup" >ה֗וּא </span><span  class="hlup" >יִשְׁלַ֤ח </span><span  class="hlup" >מַלְאָכֹו֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >פָנֶ֔יךָ </span>וְלָקַחְתָּ֥ אִשָּׁ֛ה לִבְנִ֖י מִשָּֽׁם׃ </span>|<span class="hb"><span  class="hlup" >יְהוָ֣ה׀ </span><span  class="hlup" >אֱלֹהֵ֣י </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֗יִם </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >לְקָחַ֜נִי </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >אָבִי֮ </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵ</span><span  class="hlup" >אֶ֣רֶץ </span><span  class="hlup" >מֹֽולַדְתִּי֒ </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >דִּבֶּר־</span><span  class="hlup" >לִ֜י </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֤ר </span><span  class="hlup" >נִֽשְׁבַּֽע־</span><span  class="hlup" >לִי֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹ֔ר </span><span  class="hlup" >ה֗וּא </span><span  class="hlup" >יִשְׁלַ֤ח </span><span  class="hlup" >מַלְאָכֹו֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >פָנֶ֔יךָ </span></span>|<span class="hb"><span  class="hlup" >יְהוָ֣ה׀ </span><span  class="hlup" >אֱלֹהֵ֣י </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֗יִם </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >לְקָחַ֜נִי </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >אָבִי֮ </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵ</span><span  class="hlup" >אֶ֣רֶץ </span><span  class="hlup" >מֹֽולַדְתִּי֒ </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >דִּבֶּר־</span><span  class="hlup" >לִ֜י </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲשֶׁ֤ר </span><span  class="hlup" >נִֽשְׁבַּֽע־</span><span  class="hlup" >לִי֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹ֔ר </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 24:14">Genesis 24:14</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:14" sec="Genesis 24:14">Genesis 24:14</a>וְהָיָ֣ה <span  class="hlup" >הַֽ</span><span  class="hlup" >נַּעֲרָ֗ </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >אֹמַ֤ר </span><span  class="hlup" >אֵלֶ֨יהָ֙ </span>הַטִּי־נָ֤א כַדֵּךְ֙ וְאֶשְׁתֶּ֔ה <span  class="hlup" >וְ</span><span  class="hlup" >אָמְרָ֣ה </span>שְׁתֵ֔ה וְגַם־גְּמַלֶּ֖יךָ אַשְׁקֶ֑ה <span  class="hlup" >אֹתָ֤הּ </span><span  class="hlup" >הֹכַ֨חְתָּ֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >עַבְדְּךָ֣ </span><span  class="hlup" >לְ</span><span  class="hlup" >יִצְחָ֔ק </span>וּבָ֣הּ אֵדַ֔ע כִּי־עָשִׂ֥יתָ חֶ֖סֶד עִם־אֲדֹנִֽי׃ </span>|<span class="hb"><span  class="hlup" >הַֽ</span><span  class="hlup" >נַּעֲרָ֗ </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >אֹמַ֤ר </span><span  class="hlup" >אֵלֶ֨יהָ֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >אָמְרָ֣ה </span><span  class="hlup" >אֹתָ֤הּ </span><span  class="hlup" >הֹכַ֨חְתָּ֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >עַבְדְּךָ֣ </span><span  class="hlup" >לְ</span><span  class="hlup" >יִצְחָ֔ק </span></span>|<span class="hb"><span  class="hlup" >הַֽ</span><span  class="hlup" >נַּעֲרָ֗ </span><span  class="hlup" >אֲשֶׁ֨ר </span><span  class="hlup" >אֹמַ֤ר </span><span  class="hlup" >אֵלֶ֨יהָ֙ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=49&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 24:49">Genesis 24:49</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=49&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:49" sec="Genesis 24:49">Genesis 24:49</a>וְ֠עַתָּה <span  class="hlup" >אִם־</span><span  class="hlup" >יֶשְׁכֶ֨ם </span><span  class="hlup" >עֹשִׂ֜ים </span><span  class="hlup" >חֶ֧סֶד </span><span  class="hlup" >וֶֽ</span><span  class="hlup" >אֱמֶ֛ת </span><span  class="hlup" >אֶת־</span><span  class="hlup" >אֲדֹנִ֖י </span>הַגִּ֣ידוּ לִ֑י <span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֕א </span>הַגִּ֣ידוּ לִ֔י וְאֶפְנֶ֥ה עַל־יָמִ֖ין אֹ֥ו עַל־שְׂמֹֽאל׃ </span>|<span class="hb"><span  class="hlup" >אִם־</span><span  class="hlup" >יֶשְׁכֶ֨ם </span><span  class="hlup" >עֹשִׂ֜ים </span><span  class="hlup" >חֶ֧סֶד </span><span  class="hlup" >וֶֽ</span><span  class="hlup" >אֱמֶ֛ת </span><span  class="hlup" >אֶת־</span><span  class="hlup" >אֲדֹנִ֖י </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֕א </span></span>|<span class="hb"><span  class="hlup" >אִם־</span><span  class="hlup" >יֶשְׁכֶ֨ם </span><span  class="hlup" >עֹשִׂ֜ים </span><span  class="hlup" >חֶ֧סֶד </span><span  class="hlup" >וֶֽ</span><span  class="hlup" >אֱמֶ֛ת </span><span  class="hlup" >אֶת־</span><span  class="hlup" >אֲדֹנִ֖י </span></span>



**verse** *1*



  0.51s Done


Now in the common domain.

In [25]:
indent(reset=True)
query = '''
verse
  s:common otype=sentence
    =: sa:sentence_atom
  s ## sa
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  0.93s 16 results


n | p | verse | sentence | sentence_atom
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=23&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Numbers 23:19">Numbers 23:19</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=23&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 23:19" sec="Numbers 23:19">Numbers 23:19</a>לֹ֣א אִ֥ישׁ אֵל֙ וִֽיכַזֵּ֔ב וּבֶן־אָדָ֖ם וְיִתְנֶחָ֑ם <span  class="hlup" >הַ</span><span  class="hlup" >ה֤וּא </span><span  class="hlup" >אָמַר֙ </span>וְלֹ֣א יַעֲשֶׂ֔ה <span  class="hlup" >וְ</span><span  class="hlup" >דִבֶּ֖ר </span>וְלֹ֥א יְקִימֶֽנָּה׃ </span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >ה֤וּא </span><span  class="hlup" >אָמַר֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >דִבֶּ֖ר </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >ה֤וּא </span><span  class="hlup" >אָמַר֙ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Judices&chapter=7&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Judges 7:4">Judges 7:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Judices&chapter=7&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Judges 7:4" sec="Judges 7:4">Judges 7:4</a>וַיֹּ֨אמֶר יְהוָ֜ה אֶל־גִּדְעֹ֗ון עֹוד֮ הָעָ֣ם רָב֒ הֹורֵ֤ד אֹותָם֙ אֶל־הַמַּ֔יִם וְאֶצְרְפֶ֥נּוּ לְךָ֖ שָׁ֑ם וְהָיָ֡ה <span  class="hlup" >אֲשֶׁר֩ </span><span  class="hlup" >אֹמַ֨ר </span><span  class="hlup" >אֵלֶ֜יךָ </span>זֶ֣ה׀ יֵלֵ֣ךְ אִתָּ֗ךְ <span  class="hlup" >ה֚וּא </span><span  class="hlup" >יֵלֵ֣ךְ </span><span  class="hlup" >אִתָּ֔ךְ </span>וְכֹ֨ל אֲשֶׁר־אֹמַ֜ר אֵלֶ֗יךָ זֶ֚ה לֹא־יֵלֵ֣ךְ עִמָּ֔ךְ ה֖וּא לֹ֥א יֵלֵֽךְ׃ </span>|<span class="hb"><span  class="hlup" >אֲשֶׁר֩ </span><span  class="hlup" >אֹמַ֨ר </span><span  class="hlup" >אֵלֶ֜יךָ </span><span  class="hlup" >ה֚וּא </span><span  class="hlup" >יֵלֵ֣ךְ </span><span  class="hlup" >אִתָּ֔ךְ </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁר֩ </span><span  class="hlup" >אֹמַ֨ר </span><span  class="hlup" >אֵלֶ֜יךָ </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Judices&chapter=7&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Judges 7:4">Judges 7:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Judices&chapter=7&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Judges 7:4" sec="Judges 7:4">Judges 7:4</a>וַיֹּ֨אמֶר יְהוָ֜ה אֶל־גִּדְעֹ֗ון עֹוד֮ הָעָ֣ם רָב֒ הֹורֵ֤ד אֹותָם֙ אֶל־הַמַּ֔יִם וְאֶצְרְפֶ֥נּוּ לְךָ֖ שָׁ֑ם וְהָיָ֡ה אֲשֶׁר֩ אֹמַ֨ר אֵלֶ֜יךָ זֶ֣ה׀ יֵלֵ֣ךְ אִתָּ֗ךְ ה֚וּא יֵלֵ֣ךְ אִתָּ֔ךְ <span  class="hlup" >וְ</span><span  class="hlup" >כֹ֨ל </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >אֹמַ֜ר </span><span  class="hlup" >אֵלֶ֗יךָ </span>זֶ֚ה לֹא־יֵלֵ֣ךְ עִמָּ֔ךְ <span  class="hlup" >ה֖וּא </span><span  class="hlup" >לֹ֥א </span><span  class="hlup" >יֵלֵֽךְ׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >כֹ֨ל </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >אֹמַ֜ר </span><span  class="hlup" >אֵלֶ֗יךָ </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >לֹ֥א </span><span  class="hlup" >יֵלֵֽךְ׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >כֹ֨ל </span><span  class="hlup" >אֲשֶׁר־</span><span  class="hlup" >אֹמַ֜ר </span><span  class="hlup" >אֵלֶ֗יךָ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=5&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="2_Samuel 5:6">2_Samuel 5:6</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=5&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 5:6" sec="2_Samuel 5:6">2_Samuel 5:6</a>וַיֵּ֨לֶךְ הַמֶּ֤לֶךְ וַֽאֲנָשָׁיו֙ יְר֣וּשָׁלִַ֔ם אֶל־הַיְבֻסִ֖י יֹושֵׁ֣ב הָאָ֑רֶץ <span  class="hlup" >וַ</span><span  class="hlup" >יֹּ֨אמֶר </span><span  class="hlup" >לְ</span><span  class="hlup" >דָוִ֤ד </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹר֙ </span>לֹא־תָבֹ֣וא הֵ֔נָּה כִּ֣י אִם־הֱסִֽירְךָ֗ הַעִוְרִ֤ים וְהַפִּסְחִים֙ <span  class="hlup" >לֵ</span><span  class="hlup" >אמֹ֔ר </span>לֹֽא־יָבֹ֥וא דָוִ֖ד הֵֽנָּה׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יֹּ֨אמֶר </span><span  class="hlup" >לְ</span><span  class="hlup" >דָוִ֤ד </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹר֙ </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹ֔ר </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יֹּ֨אמֶר </span><span  class="hlup" >לְ</span><span  class="hlup" >דָוִ֤ד </span><span  class="hlup" >לֵ</span><span  class="hlup" >אמֹר֙ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=21&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="2_Samuel 21:4">2_Samuel 21:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=21&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 21:4" sec="2_Samuel 21:4">2_Samuel 21:4</a>וַיֹּ֧אמְרוּ לֹ֣ו הַגִּבְעֹנִ֗ים אֵֽין־לָ֜נוּ כֶּ֤סֶף וְזָהָב֙ עִם־שָׁא֣וּל וְעִם־בֵּיתֹ֔ו וְאֵֽין־לָ֥נוּ אִ֖ישׁ לְהָמִ֣ית בְּיִשְׂרָאֵ֑ל וַיֹּ֛אמֶר <span  class="hlup" >מָֽה־</span>אַתֶּ֥ם אֹמְרִ֖ים <span  class="hlup" >אֶעֱשֶׂ֥ה </span><span  class="hlup" >לָכֶֽם׃ </span></span>|<span class="hb"><span  class="hlup" >מָֽה־</span><span  class="hlup" >אֶעֱשֶׂ֥ה </span><span  class="hlup" >לָכֶֽם׃ </span></span>|<span class="hb"><span  class="hlup" >מָֽה־</span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=30&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Isaiah 30:16">Isaiah 30:16</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=30&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 30:16" sec="Isaiah 30:16">Isaiah 30:16</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּ֨אמְר֥וּ </span>לֹא־כִ֛י עַל־ס֥וּס נָנ֖וּס עַל־כֵּ֣ן תְּנוּס֑וּן <span  class="hlup" >וְ</span>עַל־קַ֣ל נִרְכָּ֔ב עַל־כֵּ֖ן יִקַּ֥לּוּ רֹדְפֵיכֶֽם׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּ֨אמְר֥וּ </span><span  class="hlup" >וְ</span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּ֨אמְר֥וּ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=41&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Isaiah 41:17">Isaiah 41:17</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=41&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 41:17" sec="Isaiah 41:17">Isaiah 41:17</a>הָעֲנִיִּ֨ים וְהָאֶבְיֹונִ֜ים מְבַקְשִׁ֥ים מַ֨יִם֙ וָאַ֔יִן לְשֹׁונָ֖ם בַּצָּמָ֣א נָשָׁ֑תָּה <span  class="hlup" >אֲנִ֤י </span><span  class="hlup" >יְהוָה֙ </span>אֶעֱנֵ֔ם <span  class="hlup" >אֱלֹהֵ֥י </span><span  class="hlup" >יִשְׂרָאֵ֖ל </span>לֹ֥א אֶעֶזְבֵֽם׃ </span>|<span class="hb"><span  class="hlup" >אֲנִ֤י </span><span  class="hlup" >יְהוָה֙ </span><span  class="hlup" >אֱלֹהֵ֥י </span><span  class="hlup" >יִשְׂרָאֵ֖ל </span></span>|<span class="hb"><span  class="hlup" >אֲנִ֤י </span><span  class="hlup" >יְהוָה֙ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=41&verse=28&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Isaiah 41:28">Isaiah 41:28</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=41&verse=28&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 41:28" sec="Isaiah 41:28">Isaiah 41:28</a><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֨רֶא֙ </span>וְאֵ֣ין אִ֔ישׁ <span  class="hlup" >וּ</span><span  class="hlup" >מֵ</span><span  class="hlup" >אֵ֖לֶּה </span>וְאֵ֣ין יֹועֵ֑ץ וְאֶשְׁאָלֵ֖ם וְיָשִׁ֥יבוּ דָבָֽר׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֨רֶא֙ </span><span  class="hlup" >וּ</span><span  class="hlup" >מֵ</span><span  class="hlup" >אֵ֖לֶּה </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֨רֶא֙ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=55&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Isaiah 55:7">Isaiah 55:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=55&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 55:7" sec="Isaiah 55:7">Isaiah 55:7</a>יַעֲזֹ֤ב רָשָׁע֙ דַּרְכֹּ֔ו וְאִ֥ישׁ אָ֖וֶן מַחְשְׁבֹתָ֑יו <span  class="hlup" >וְ</span><span  class="hlup" >יָשֹׁ֤ב </span><span  class="hlup" >אֶל־</span><span  class="hlup" >יְהוָה֙ </span>וִֽירַחֲמֵ֔הוּ <span  class="hlup" >וְ</span><span  class="hlup" >אֶל־</span><span  class="hlup" >אֱלֹהֵ֖ינוּ </span>כִּֽי־יַרְבֶּ֥ה לִסְלֹֽוחַ׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >יָשֹׁ֤ב </span><span  class="hlup" >אֶל־</span><span  class="hlup" >יְהוָה֙ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶל־</span><span  class="hlup" >אֱלֹהֵ֖ינוּ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >יָשֹׁ֤ב </span><span  class="hlup" >אֶל־</span><span  class="hlup" >יְהוָה֙ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=65&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Isaiah 65:1">Isaiah 65:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=65&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 65:1" sec="Isaiah 65:1">Isaiah 65:1</a>נִדְרַ֨שְׁתִּי֙ לְלֹ֣וא שָׁאָ֔לוּ נִמְצֵ֖אתִי לְלֹ֣א בִקְשֻׁ֑נִי <span  class="hlup" >אָמַ֨רְתִּי֙ </span>הִנֵּ֣נִי הִנֵּ֔נִי <span  class="hlup" >אֶל־</span><span  class="hlup" >גֹּ֖וי </span><span  class="hlup" >לֹֽא־</span><span  class="hlup" >קֹרָ֥א </span><span  class="hlup" >בִ</span><span  class="hlup" >שְׁמִֽי׃ </span></span>|<span class="hb"><span  class="hlup" >אָמַ֨רְתִּי֙ </span><span  class="hlup" >אֶל־</span><span  class="hlup" >גֹּ֖וי </span><span  class="hlup" >לֹֽא־</span><span  class="hlup" >קֹרָ֥א </span><span  class="hlup" >בִ</span><span  class="hlup" >שְׁמִֽי׃ </span></span>|<span class="hb"><span  class="hlup" >אָמַ֨רְתִּי֙ </span></span>



**verse** *1*



  0.96s Done


## || disjoint slots

True when the two objects in question do not share any slots.
This is a rather loose relationship.

This can be used for locating gaps: a textual object that lies inside a gap of another object.
See also [gaps](searchGaps.ipynb).

Here we check whether there are phrases with disjoint subphrases.

In [26]:
indent(reset=True)
query = '''
verse
  p:phrase
    s1:subphrase
    < s2:subphrase
  s1 || s2
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

  1.90s 167760 results


n | p | verse | phrase | subphrase | subphrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1">Genesis 1:1</a>בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים <span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span></span>|<span class="hb"><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה <span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span>וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </span>|<span class="hb"><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >תֹ֨הוּ֙ </span></span>|<span class="hb"><span  class="hlup" >בֹ֔הוּ </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span>וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֣י </span></span>|<span class="hb"><span  class="hlup" >תְהֹ֑ום </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְ<span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span>מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </span>|<span class="hb"><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hlup" >ר֣וּחַ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֔ים </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֥י </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים <span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span></span>|<span class="hb"><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:5">Genesis 1:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:5" sec="Genesis 1:5">Genesis 1:5</a>וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר <span  class="hlup" >יֹ֥ום </span><span  class="hlup" >אֶחָֽד׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span><span  class="hlup" >אֶחָֽד׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span></span>|<span class="hb"><span  class="hlup" >אֶחָֽד׃ פ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:6">Genesis 1:6</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:6" sec="Genesis 1:6">Genesis 1:6</a>וַיֹּ֣אמֶר אֱלֹהִ֔ים יְהִ֥י רָקִ֖יעַ <span  class="hlup" >בְּ</span><span  class="hlup" >תֹ֣וךְ </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּ֑יִם </span>וִיהִ֣י מַבְדִּ֔יל בֵּ֥ין מַ֖יִם לָמָֽיִם׃ </span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >תֹ֣וךְ </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּ֑יִם </span></span>|<span class="hb"><span  class="hlup" >תֹ֣וךְ </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >מָּ֑יִם </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:8">Genesis 1:8</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:8" sec="Genesis 1:8">Genesis 1:8</a>וַיִּקְרָ֧א אֱלֹהִ֛ים לָֽרָקִ֖יעַ שָׁמָ֑יִם וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר <span  class="hlup" >יֹ֥ום </span><span  class="hlup" >שֵׁנִֽי׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span><span  class="hlup" >שֵׁנִֽי׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span></span>|<span class="hb"><span  class="hlup" >שֵׁנִֽי׃ פ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:9">Genesis 1:9</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:9" sec="Genesis 1:9">Genesis 1:9</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים יִקָּו֨וּ <span  class="hlup" >הַ</span><span  class="hlup" >מַּ֜יִם </span><span  class="hlup" >מִ</span><span  class="hlup" >תַּ֤חַת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֨יִם֙ </span>אֶל־מָקֹ֣ום אֶחָ֔ד וְתֵרָאֶ֖ה הַיַּבָּשָׁ֑ה וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֜יִם </span><span  class="hlup" >מִ</span><span  class="hlup" >תַּ֤חַת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֨יִם֙ </span></span>|<span class="hb"><span  class="hlup" >תַּ֤חַת </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֨יִם֙ </span></span>



**verse** *1*



  1.91s Done


Now in the common domain.

In [27]:
indent(reset=True)
query = '''
verse
  p:common otype=phrase
    s1:subphrase
    < s2:subphrase
  s1 || s2
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.30s 10958 results


n | p | verse | phrase | subphrase | subphrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֥י </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:5">Genesis 1:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:5" sec="Genesis 1:5">Genesis 1:5</a>וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר <span  class="hlup" >יֹ֥ום </span><span  class="hlup" >אֶחָֽד׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span><span  class="hlup" >אֶחָֽד׃ פ </span></span>|<span class="hb"><span  class="hlup" >יֹ֥ום </span></span>|<span class="hb"><span  class="hlup" >אֶחָֽד׃ פ </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">Genesis 1:29</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span>וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֣י </span></span>|<span class="hb"><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">Genesis 1:29</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span>וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֣י </span></span>|<span class="hb"><span  class="hlup" >כָל־</span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">Genesis 1:29</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span>וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>|<span class="hb"><span  class="hlup" >פְּנֵ֣י </span></span>|<span class="hb"><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">Genesis 1:29</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ <span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span>וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >כָל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>|<span class="hb"><span  class="hlup" >כָל־</span></span>|<span class="hb"><span  class="hlup" >הָ</span><span  class="hlup" >אָ֔רֶץ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:4">Genesis 2:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:4" sec="Genesis 2:4">Genesis 2:4</a>אֵ֣לֶּה תֹולְדֹ֧ות הַשָּׁמַ֛יִם וְהָאָ֖רֶץ בְּהִבָּֽרְאָ֑ם בְּיֹ֗ום עֲשֹׂ֛ות <span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span>אֶ֥רֶץ וְשָׁמָֽיִם׃ </span>|<span class="hb"><span  class="hlup" >יְהוָ֥ה </span><span  class="hlup" >אֱלֹהִ֖ים </span></span>|<span class="hb"><span  class="hlup" >יְהוָ֥ה </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֖ים </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:5">Genesis 2:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:5" sec="Genesis 2:5">Genesis 2:5</a>וְכֹ֣ל׀ שִׂ֣יחַ הַשָּׂדֶ֗ה טֶ֚רֶם יִֽהְיֶ֣ה בָאָ֔רֶץ וְכָל־עֵ֥שֶׂב הַשָּׂדֶ֖ה טֶ֣רֶם יִצְמָ֑ח כִּי֩ לֹ֨א הִמְטִ֜יר <span  class="hlup" >יְהוָ֤ה </span><span  class="hlup" >אֱלֹהִים֙ </span>עַל־הָאָ֔רֶץ וְאָדָ֣ם אַ֔יִן לַֽעֲבֹ֖ד אֶת־הָֽאֲדָמָֽה׃ </span>|<span class="hb"><span  class="hlup" >יְהוָ֤ה </span><span  class="hlup" >אֱלֹהִים֙ </span></span>|<span class="hb"><span  class="hlup" >יְהוָ֤ה </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִים֙ </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:7">Genesis 2:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:7" sec="Genesis 2:7">Genesis 2:7</a>וַיִּיצֶר֩ <span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span>אֶת־הָֽאָדָ֗ם עָפָר֙ מִן־הָ֣אֲדָמָ֔ה וַיִּפַּ֥ח בְּאַפָּ֖יו נִשְׁמַ֣ת חַיִּ֑ים וַֽיְהִ֥י הָֽאָדָ֖ם לְנֶ֥פֶשׁ חַיָּֽה׃ </span>|<span class="hb"><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֜ים </span></span>|<span class="hb"><span  class="hlup" >יְהוָ֨ה </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֜ים </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:8">Genesis 2:8</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:8" sec="Genesis 2:8">Genesis 2:8</a>וַיִּטַּ֞ע <span  class="hlup" >יְהוָ֧ה </span><span  class="hlup" >אֱלֹהִ֛ים </span>גַּן־בְּעֵ֖דֶן מִקֶּ֑דֶם וַיָּ֣שֶׂם שָׁ֔ם אֶת־הָֽאָדָ֖ם אֲשֶׁ֥ר יָצָֽר׃ </span>|<span class="hb"><span  class="hlup" >יְהוָ֧ה </span><span  class="hlup" >אֱלֹהִ֛ים </span></span>|<span class="hb"><span  class="hlup" >יְהוָ֧ה </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֛ים </span></span>



**verse** *1*



  1.31s Done


We see that Genesis 1:1 has fallen out. Let's check the ranks of the lexemes of its last phrase:

In [28]:
firstSentence = F.otype.s('sentence')[0]
lastPhrase = L.d(firstSentence, otype='phrase')[-1]
[(F.g_word_utf8.v(w), F.rank_lex.v(w)) for w in L.d(lastPhrase, otype='word')]

[('אֵ֥ת', 4),
 ('הַ', 1),
 ('שָּׁמַ֖יִם', 122),
 ('וְ', 0),
 ('אֵ֥ת', 4),
 ('הָ', 1),
 ('אָֽרֶץ', 22)]

There you have it: the heavens are rare!

## [[ and ]] embedding

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `m`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# Positioning

## << and >> before and after (slot-wise)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [29]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, condensed=False, start=1, end=1, colorMap={2: 'lightyellow', 3: 'cyan', 4: 'magenta', 5: 'blue'})

  1.97s 30933 results


n | p | verse | sentence | clause | phrase | clause
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span></span>|<span class="hb"><span  class="hlup" >עֹ֤שֶׂה </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span></span>|<span class="hb"><span  class="hlup" >פְּרִי֙ </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span></span>|<span class="hb"><span  class="hlup" >עֹ֤שֶׂה </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span></span>|<span class="hb"><span  class="hlup" >פְּרִי֙ </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >מַזְרִ֣יעַ </span><span  class="hlup" >זֶ֔רַע </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span></span>|<span class="hb"><span  class="hlup" >עֹ֥שֶׂה </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span></span>|<span class="hb"><span  class="hlup" >פְּרִ֛י </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>|<span class="hb"><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >עֹ֥שֶׂה </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>|<span class="hb"><span  class="hlup" >מַזְרִ֤יעַ </span><span  class="hlup" >זֶ֨רַע֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >פְּרִ֛י </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֥ר </span><span  class="hlup" >זַרְעֹו־</span><span  class="hlup" >בֹ֖ו </span></span>



**verse** *1*



In the common domain:

In [30]:
indent(reset=True)
query = '''
verse
  sentence
    c1:common otype=clause
    p:rare otype=phrase
    c2:common otype=clause
    c1 << p
    c2 >> p
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  2.26s 236 results


n | p | verse | sentence | clause | phrase | clause
--- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:5">Genesis 3:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:5" sec="Genesis 3:5"><span  class="hlup" >Genesis 3:5</span></a><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hlup" >נִפְקְח֖וּ </span></span>|<span class="hb"><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:5">Genesis 3:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:5" sec="Genesis 3:5"><span  class="hlup" >Genesis 3:5</span></a><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span></span>|<span class="hb"><span  class="hlup" >נִפְקְח֖וּ </span></span>|<span class="hb"><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:5">Genesis 3:5</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:5" sec="Genesis 3:5"><span  class="hlup" >Genesis 3:5</span></a><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֚י </span><span  class="hlup" >יֹדֵ֣עַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹום֙ </span><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span><span  class="hlup" >וְ</span><span  class="hlup" >נִפְקְח֖וּ </span><span  class="hlup" >עֵֽינֵיכֶ֑ם </span><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span><span  class="hlup" >יֹדְעֵ֖י </span><span  class="hlup" >טֹ֥וב </span><span  class="hlup" >וָ</span><span  class="hlup" >רָֽע׃ </span></span>|<span class="hb"><span  class="hlup" >אֲכָלְכֶ֣ם </span><span  class="hlup" >מִמֶּ֔נּוּ </span></span>|<span class="hb"><span  class="hlup" >נִפְקְח֖וּ </span></span>|<span class="hb"><span  class="hlup" >וִ</span><span  class="hlup" >הְיִיתֶם֙ </span><span  class="hlup" >כֵּֽ</span><span  class="hlup" >אלֹהִ֔ים </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 3:12">Genesis 3:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:12" sec="Genesis 3:12">Genesis 3:12</a>וַיֹּ֖אמֶר הָֽאָדָ֑ם <span  class="hlup" >הָֽ</span><span  class="hlup" >אִשָּׁה֙ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >נָתַ֣תָּה </span><span  class="hlup" >עִמָּדִ֔י </span><span  class="hlup" >הִ֛וא </span><span  class="hlup" >נָֽתְנָה־</span><span  class="hlup" >לִּ֥י </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >וָ</span><span  class="hlup" >אֹכֵֽל׃ </span></span>|<span class="hb"><span  class="hlup" >הָֽ</span><span  class="hlup" >אִשָּׁה֙ </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >נָתַ֣תָּה </span><span  class="hlup" >עִמָּדִ֔י </span><span  class="hlup" >הִ֛וא </span><span  class="hlup" >נָֽתְנָה־</span><span  class="hlup" >לִּ֥י </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֖ץ </span><span  class="hlup" >וָ</span><span  class="hlup" >אֹכֵֽל׃ </span></span>|<span class="hb"><span  class="hlup" >הָֽ</span><span  class="hlup" >אִשָּׁה֙ </span></span>|<span class="hb"><span  class="hlup" >עִמָּדִ֔י </span></span>|<span class="hb"><span  class="hlup" >וָ</span><span  class="hlup" >אֹכֵֽל׃ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 14:17">Genesis 14:17</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=14&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 14:17" sec="Genesis 14:17">Genesis 14:17</a><span  class="hlup" >וַ</span><span  class="hlup" >יֵּצֵ֣א </span><span  class="hlup" >מֶֽלֶךְ־</span><span  class="hlup" >סְדֹם֮ </span><span  class="hlup" >לִ</span><span  class="hlup" >קְרָאתֹו֒ </span><span  class="hlup" >אַחֲרֵ֣י </span><span  class="hlup" >שׁוּבֹ֗ו </span><span  class="hlup" >מֵֽ</span><span  class="hlup" >הַכֹּות֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כְּדָר־לָעֹ֔מֶר </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מְּלָכִ֖ים </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >אִתֹּ֑ו </span><span  class="hlup" >אֶל־</span><span  class="hlup" >עֵ֣מֶק </span><span  class="hlup" >שָׁוֵ֔ה </span>ה֖וּא עֵ֥מֶק הַמֶּֽלֶךְ׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יֵּצֵ֣א </span><span  class="hlup" >מֶֽלֶךְ־</span><span  class="hlup" >סְדֹם֮ </span><span  class="hlup" >לִ</span><span  class="hlup" >קְרָאתֹו֒ </span><span  class="hlup" >אַחֲרֵ֣י </span><span  class="hlup" >שׁוּבֹ֗ו </span><span  class="hlup" >מֵֽ</span><span  class="hlup" >הַכֹּות֙ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כְּדָר־לָעֹ֔מֶר </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מְּלָכִ֖ים </span><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >אִתֹּ֑ו </span><span  class="hlup" >אֶל־</span><span  class="hlup" >עֵ֣מֶק </span><span  class="hlup" >שָׁוֵ֔ה </span></span>|<span class="hb"><span  class="hlup" >אַחֲרֵ֣י </span><span  class="hlup" >שׁוּבֹ֗ו </span></span>|<span class="hb"><span  class="hlup" >אֶת־</span><span  class="hlup" >כְּדָר־לָעֹ֔מֶר </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >מְּלָכִ֖ים </span></span>|<span class="hb"><span  class="hlup" >אֲשֶׁ֣ר </span><span  class="hlup" >אִתֹּ֑ו </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=15&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 15:7">Genesis 15:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=15&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 15:7" sec="Genesis 15:7">Genesis 15:7</a>וַיֹּ֖אמֶר אֵלָ֑יו <span  class="hlup" >אֲנִ֣י </span><span  class="hlup" >יְהוָ֗ה </span><span  class="hlup" >אֲשֶׁ֤ר </span><span  class="hlup" >הֹוצֵאתִ֨יךָ֙ </span><span  class="hlup" >מֵ</span><span  class="hlup" >א֣וּר </span><span  class="hlup" >כַּשְׂדִּ֔ים </span><span  class="hlup" >לָ֧</span><span  class="hlup" >תֶת </span><span  class="hlup" >לְךָ֛ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֥רֶץ </span><span  class="hlup" >הַ</span><span  class="hlup" >זֹּ֖את </span><span  class="hlup" >לְ</span><span  class="hlup" >רִשְׁתָּֽהּ׃ </span></span>|<span class="hb"><span  class="hlup" >אֲנִ֣י </span><span  class="hlup" >יְהוָ֗ה </span><span  class="hlup" >אֲשֶׁ֤ר </span><span  class="hlup" >הֹוצֵאתִ֨יךָ֙ </span><span  class="hlup" >מֵ</span><span  class="hlup" >א֣וּר </span><span  class="hlup" >כַּשְׂדִּ֔ים </span><span  class="hlup" >לָ֧</span><span  class="hlup" >תֶת </span><span  class="hlup" >לְךָ֛ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֥רֶץ </span><span  class="hlup" >הַ</span><span  class="hlup" >זֹּ֖את </span><span  class="hlup" >לְ</span><span  class="hlup" >רִשְׁתָּֽהּ׃ </span></span>|<span class="hb"><span  class="hlup" >אֲנִ֣י </span><span  class="hlup" >יְהוָ֗ה </span></span>|<span class="hb"><span  class="hlup" >מֵ</span><span  class="hlup" >א֣וּר </span><span  class="hlup" >כַּשְׂדִּ֔ים </span></span>|<span class="hb"><span  class="hlup" >לָ֧</span><span  class="hlup" >תֶת </span><span  class="hlup" >לְךָ֛ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֥רֶץ </span><span  class="hlup" >הַ</span><span  class="hlup" >זֹּ֖את </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 18:21">Genesis 18:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 18:21" sec="Genesis 18:21">Genesis 18:21</a>אֵֽרֲדָה־נָּ֣א <span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span>אֵדָֽעָה׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span></span>|<span class="hb"><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 18:21">Genesis 18:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 18:21" sec="Genesis 18:21">Genesis 18:21</a>אֵֽרֲדָה־נָּ֣א <span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span>אֵדָֽעָה׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span></span>|<span class="hb"><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 18:21">Genesis 18:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 18:21" sec="Genesis 18:21">Genesis 18:21</a>אֵֽרֲדָה־נָּ֣א <span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span>אֵדָֽעָה׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span></span>|<span class="hb"><span  class="hlup" >כָּלָ֑ה </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 18:21">Genesis 18:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=18&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 18:21" sec="Genesis 18:21">Genesis 18:21</a>אֵֽרֲדָה־נָּ֣א <span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span>אֵדָֽעָה׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶרְאֶ֔ה </span><span  class="hlup" >הַ</span><span  class="hlup" >כְּ</span><span  class="hlup" >צַעֲקָתָ֛הּ </span><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span><span  class="hlup" >עָשׂ֣וּ׀ </span><span  class="hlup" >כָּלָ֑ה </span><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>|<span class="hb"><span  class="hlup" >הַ</span><span  class="hlup" >בָּ֥אָה </span><span  class="hlup" >אֵלַ֖י </span></span>|<span class="hb"><span  class="hlup" >כָּלָ֑ה </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אִם־</span><span  class="hlup" >לֹ֖א </span></span>



**verse** *1*



  2.28s Done


## =: same start slots
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

## := same end slots
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

## :: same boundary slots
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

Let us look for a phrase, whose start and end slots coincide with its containing clause.
But only if the phrase does not conincide with its parent clause.

In [31]:
query = '''
verse
    c:clause
        :: p:phrase
c ## p
'''
results = A.search(query)
A.table(results, start=1, end=10)
A.show(results, start=1, end=5, condenseType='clause')

  0.40s 54 results


n | p | verse | clause | phrase
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 10:21">Genesis 10:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21" sec="Genesis 10:21">Genesis 10:21</a>וּלְשֵׁ֥ם יֻלַּ֖ד <span  class="hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲבִי֙ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >בְּנֵי־</span><span  class="hlup" >עֵ֔בֶר </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>|<span class="hb"><span  class="hlup" >גַּם־</span><span  class="hlup" >ה֑וּא </span><span  class="hlup" >אֲחִ֖י </span><span  class="hlup" >יֶ֥פֶת </span><span  class="hlup" >הַ</span><span  class="hlup" >גָּדֹֽול׃ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 24:24">Genesis 24:24</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:24" sec="Genesis 24:24">Genesis 24:24</a>וַתֹּ֣אמֶר אֵלָ֔יו <span  class="hlup" >בַּת־</span><span  class="hlup" >בְּתוּאֵ֖ל </span><span  class="hlup" >אָנֹ֑כִי </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >מִלְכָּ֕ה </span>אֲשֶׁ֥ר יָלְדָ֖ה לְנָחֹֽור׃ </span>|<span class="hb"><span  class="hlup" >בַּת־</span><span  class="hlup" >בְּתוּאֵ֖ל </span><span  class="hlup" >אָנֹ֑כִי </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >מִלְכָּ֕ה </span></span>|<span class="hb"><span  class="hlup" >בַּת־</span><span  class="hlup" >בְּתוּאֵ֖ל </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >מִלְכָּ֕ה </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 31:16">Genesis 31:16</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16" sec="Genesis 31:16">Genesis 31:16</a>כִּ֣י כָל־הָעֹ֗שֶׁר אֲשֶׁ֨ר הִצִּ֤יל אֱלֹהִים֙ מֵֽאָבִ֔ינוּ <span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span>וְעַתָּ֗ה כֹּל֩ אֲשֶׁ֨ר אָמַ֧ר אֱלֹהִ֛ים אֵלֶ֖יךָ עֲשֵֽׂה׃ </span>|<span class="hb"><span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span></span>|<span class="hb"><span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=53&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 31:53">Genesis 31:53</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=53&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:53" sec="Genesis 31:53">Genesis 31:53</a><span  class="hlup" >אֱלֹהֵ֨י </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >וֵֽ</span><span  class="hlup" >אלֹהֵ֤י </span><span  class="hlup" >נָחֹור֙ </span><span  class="hlup" >יִשְׁפְּט֣וּ </span><span  class="hlup" >בֵינֵ֔ינוּ </span><span  class="hlup" >אֱלֹהֵ֖י </span><span  class="hlup" >אֲבִיהֶ֑ם </span>וַיִּשָּׁבַ֣ע יַעֲקֹ֔ב בְּפַ֖חַד אָבִ֥יו יִצְחָֽק׃ </span>|<span class="hb"><span  class="hlup" >אֱלֹהֵ֨י </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >וֵֽ</span><span  class="hlup" >אלֹהֵ֤י </span><span  class="hlup" >נָחֹור֙ </span><span  class="hlup" >יִשְׁפְּט֣וּ </span><span  class="hlup" >בֵינֵ֔ינוּ </span><span  class="hlup" >אֱלֹהֵ֖י </span><span  class="hlup" >אֲבִיהֶ֑ם </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהֵ֨י </span><span  class="hlup" >אַבְרָהָ֜ם </span><span  class="hlup" >וֵֽ</span><span  class="hlup" >אלֹהֵ֤י </span><span  class="hlup" >נָחֹור֙ </span><span  class="hlup" >אֱלֹהֵ֖י </span><span  class="hlup" >אֲבִיהֶ֑ם </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 28:1">Exodus 28:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:1" sec="Exodus 28:1">Exodus 28:1</a>וְאַתָּ֡ה הַקְרֵ֣ב אֵלֶיךָ֩ אֶת־אַהֲרֹ֨ן אָחִ֜יךָ וְאֶת־בָּנָ֣יו אִתֹּ֔ו מִתֹּ֛וךְ בְּנֵ֥י יִשְׂרָאֵ֖ל <span  class="hlup" >לְ</span><span  class="hlup" >כַהֲנֹו־</span><span  class="hlup" >לִ֑י </span><span  class="hlup" >אַהֲרֹ֕ן </span><span  class="hlup" >נָדָ֧ב </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲבִיה֛וּא </span><span  class="hlup" >אֶלְעָזָ֥ר </span><span  class="hlup" >וְ</span><span  class="hlup" >אִיתָמָ֖ר </span><span  class="hlup" >בְּנֵ֥י </span><span  class="hlup" >אַהֲרֹֽן׃ </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >כַהֲנֹו־</span><span  class="hlup" >לִ֑י </span><span  class="hlup" >אַהֲרֹ֕ן </span><span  class="hlup" >נָדָ֧ב </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲבִיה֛וּא </span><span  class="hlup" >אֶלְעָזָ֥ר </span><span  class="hlup" >וְ</span><span  class="hlup" >אִיתָמָ֖ר </span><span  class="hlup" >בְּנֵ֥י </span><span  class="hlup" >אַהֲרֹֽן׃ </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >כַהֲנֹו־</span><span  class="hlup" >אַהֲרֹ֕ן </span><span  class="hlup" >נָדָ֧ב </span><span  class="hlup" >וַ</span><span  class="hlup" >אֲבִיה֛וּא </span><span  class="hlup" >אֶלְעָזָ֥ר </span><span  class="hlup" >וְ</span><span  class="hlup" >אִיתָמָ֖ר </span><span  class="hlup" >בְּנֵ֥י </span><span  class="hlup" >אַהֲרֹֽן׃ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 28:14">Exodus 28:14</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:14" sec="Exodus 28:14">Exodus 28:14</a>וּשְׁתֵּ֤י שַׁרְשְׁרֹת֙ זָהָ֣ב טָהֹ֔ור <span  class="hlup" >מִגְבָּלֹ֛ת </span><span  class="hlup" >תַּעֲשֶׂ֥ה </span><span  class="hlup" >אֹתָ֖ם </span><span  class="hlup" >מַעֲשֵׂ֣ה </span><span  class="hlup" >עֲבֹ֑ת </span>וְנָתַתָּ֛ה אֶת־שַׁרְשְׁרֹ֥ת הָעֲבֹתֹ֖ת עַל־הַֽמִּשְׁבְּצֹֽת׃ ס </span>|<span class="hb"><span  class="hlup" >מִגְבָּלֹ֛ת </span><span  class="hlup" >תַּעֲשֶׂ֥ה </span><span  class="hlup" >אֹתָ֖ם </span><span  class="hlup" >מַעֲשֵׂ֣ה </span><span  class="hlup" >עֲבֹ֑ת </span></span>|<span class="hb"><span  class="hlup" >מִגְבָּלֹ֛ת </span><span  class="hlup" >מַעֲשֵׂ֣ה </span><span  class="hlup" >עֲבֹ֑ת </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=29&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 29:18">Exodus 29:18</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=29&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 29:18" sec="Exodus 29:18">Exodus 29:18</a>וְהִקְטַרְתָּ֤ אֶת־כָּל־הָאַ֨יִל֙ הַמִּזְבֵּ֔חָה <span  class="hlup" >עֹלָ֥ה </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >לַֽ</span><span  class="hlup" >יהוָ֑ה </span><span  class="hlup" >רֵ֣יחַ </span><span  class="hlup" >נִיחֹ֔וחַ </span>אִשֶּׁ֥ה לַיהוָ֖ה הֽוּא׃ </span>|<span class="hb"><span  class="hlup" >עֹלָ֥ה </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >לַֽ</span><span  class="hlup" >יהוָ֑ה </span><span  class="hlup" >רֵ֣יחַ </span><span  class="hlup" >נִיחֹ֔וחַ </span></span>|<span class="hb"><span  class="hlup" >עֹלָ֥ה </span><span  class="hlup" >רֵ֣יחַ </span><span  class="hlup" >נִיחֹ֔וחַ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 30:4">Exodus 30:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:4" sec="Exodus 30:4">Exodus 30:4</a>וּשְׁתֵּי֩ טַבְּעֹ֨ת זָהָ֜ב תַּֽעֲשֶׂה־לֹּ֣ו׀ מִתַּ֣חַת לְזֵרֹ֗ו <span  class="hlup" >עַ֚ל </span><span  class="hlup" >שְׁתֵּ֣י </span><span  class="hlup" >צַלְעֹתָ֔יו </span><span  class="hlup" >תַּעֲשֶׂ֖ה </span><span  class="hlup" >עַל־</span><span  class="hlup" >שְׁנֵ֣י </span><span  class="hlup" >צִדָּ֑יו </span>וְהָיָה֙ לְבָתִּ֣ים לְבַדִּ֔ים לָשֵׂ֥את אֹתֹ֖ו בָּהֵֽמָּה׃ </span>|<span class="hb"><span  class="hlup" >עַ֚ל </span><span  class="hlup" >שְׁתֵּ֣י </span><span  class="hlup" >צַלְעֹתָ֔יו </span><span  class="hlup" >תַּעֲשֶׂ֖ה </span><span  class="hlup" >עַל־</span><span  class="hlup" >שְׁנֵ֣י </span><span  class="hlup" >צִדָּ֑יו </span></span>|<span class="hb"><span  class="hlup" >עַ֚ל </span><span  class="hlup" >שְׁתֵּ֣י </span><span  class="hlup" >צַלְעֹתָ֔יו </span><span  class="hlup" >עַל־</span><span  class="hlup" >שְׁנֵ֣י </span><span  class="hlup" >צִדָּ֑יו </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 30:13">Exodus 30:13</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:13" sec="Exodus 30:13">Exodus 30:13</a><span  class="hlup" >זֶ֣ה׀ </span><span  class="hlup" >יִתְּנ֗וּ </span><span  class="hlup" >כָּל־</span>הָעֹבֵר֙ עַל־הַפְּקֻדִ֔ים <span  class="hlup" >מַחֲצִ֥ית </span><span  class="hlup" >הַ</span><span  class="hlup" >שֶּׁ֖קֶל </span><span  class="hlup" >בְּ</span><span  class="hlup" >שֶׁ֣קֶל </span><span  class="hlup" >הַ</span><span  class="hlup" >קֹּ֑דֶשׁ </span>עֶשְׂרִ֤ים גֵּרָה֙ הַשֶּׁ֔קֶל מַחֲצִ֣ית הַשֶּׁ֔קֶל תְּרוּמָ֖ה לַֽיהוָֽה׃ </span>|<span class="hb"><span  class="hlup" >זֶ֣ה׀ </span><span  class="hlup" >יִתְּנ֗וּ </span><span  class="hlup" >כָּל־</span><span  class="hlup" >מַחֲצִ֥ית </span><span  class="hlup" >הַ</span><span  class="hlup" >שֶּׁ֖קֶל </span><span  class="hlup" >בְּ</span><span  class="hlup" >שֶׁ֣קֶל </span><span  class="hlup" >הַ</span><span  class="hlup" >קֹּ֑דֶשׁ </span></span>|<span class="hb"><span  class="hlup" >זֶ֣ה׀ </span><span  class="hlup" >מַחֲצִ֥ית </span><span  class="hlup" >הַ</span><span  class="hlup" >שֶּׁ֖קֶל </span><span  class="hlup" >בְּ</span><span  class="hlup" >שֶׁ֣קֶל </span><span  class="hlup" >הַ</span><span  class="hlup" >קֹּ֑דֶשׁ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=36&verse=35&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Exodus 36:35">Exodus 36:35</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=36&verse=35&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 36:35" sec="Exodus 36:35">Exodus 36:35</a>וַיַּ֨עַשׂ֙ אֶת־הַפָּרֹ֔כֶת תְּכֵ֧לֶת וְאַרְגָּמָ֛ן וְתֹולַ֥עַת שָׁנִ֖י וְשֵׁ֣שׁ מָשְׁזָ֑ר <span  class="hlup" >מַעֲשֵׂ֥ה </span><span  class="hlup" >חֹשֵׁ֛ב </span><span  class="hlup" >עָשָׂ֥ה </span><span  class="hlup" >אֹתָ֖הּ </span><span  class="hlup" >כְּרֻבִֽים׃ </span></span>|<span class="hb"><span  class="hlup" >מַעֲשֵׂ֥ה </span><span  class="hlup" >חֹשֵׁ֛ב </span><span  class="hlup" >עָשָׂ֥ה </span><span  class="hlup" >אֹתָ֖הּ </span><span  class="hlup" >כְּרֻבִֽים׃ </span></span>|<span class="hb"><span  class="hlup" >מַעֲשֵׂ֥ה </span><span  class="hlup" >חֹשֵׁ֛ב </span><span  class="hlup" >כְּרֻבִֽים׃ </span></span>



**verse** *1*





**verse** *2*





**verse** *3*





**verse** *4*





**verse** *5*



Here you see an extra phrase in such clauses, lying embedded in the clause-spanning phrase.

A nice case of **Mind the gap!**.

In the common domain:

In [32]:
indent(reset=True)
query = '''
verse
    c:common otype=clause
        :: p:common otype=phrase
c ## p
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.65s 3 results


n | p | verse | clause | phrase
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 31:16">Genesis 31:16</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16" sec="Genesis 31:16">Genesis 31:16</a>כִּ֣י כָל־הָעֹ֗שֶׁר אֲשֶׁ֨ר הִצִּ֤יל אֱלֹהִים֙ מֵֽאָבִ֔ינוּ <span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span>וְעַתָּ֗ה כֹּל֩ אֲשֶׁ֨ר אָמַ֧ר אֱלֹהִ֛ים אֵלֶ֖יךָ עֲשֵֽׂה׃ </span>|<span class="hb"><span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >ה֖וּא </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span></span>|<span class="hb"><span  class="hlup" >לָ֥נוּ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >בָנֵ֑ינוּ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=11&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Leviticus 11:9">Leviticus 11:9</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Leviticus&chapter=11&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Leviticus 11:9" sec="Leviticus 11:9">Leviticus 11:9</a><span  class="hlup" >אֶת־</span><span  class="hlup" >זֶה֙ </span><span  class="hlup" >תֹּֽאכְל֔וּ </span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּ֖ל </span>אֲשֶׁ֣ר בַּמָּ֑יִם כֹּ֣ל אֲשֶׁר־לֹו֩ סְנַפִּ֨יר וְקַשְׂקֶ֜שֶׂת בַּמַּ֗יִם בַּיַּמִּ֛ים וּבַנְּחָלִ֖ים אֹתָ֥ם תֹּאכֵֽלוּ׃ </span>|<span class="hb"><span  class="hlup" >אֶת־</span><span  class="hlup" >זֶה֙ </span><span  class="hlup" >תֹּֽאכְל֔וּ </span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּ֖ל </span></span>|<span class="hb"><span  class="hlup" >אֶת־</span><span  class="hlup" >זֶה֙ </span><span  class="hlup" >מִ</span><span  class="hlup" >כֹּ֖ל </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Ezechiel&chapter=14&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Ezekiel 14:4">Ezekiel 14:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Ezechiel&chapter=14&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Ezekiel 14:4" sec="Ezekiel 14:4">Ezekiel 14:4</a>לָכֵ֣ן דַּבֵּר־אֹ֠ותָם וְאָמַרְתָּ֨ אֲלֵיהֶ֜ם כֹּה־אָמַ֣ר׀ אֲדֹנָ֣י יְהוִ֗ה <span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >יִשְׂרָאֵ֡ל </span>אֲשֶׁר֩ יַעֲלֶ֨ה אֶת־גִּלּוּלָ֜יו אֶל־לִבֹּ֗ו וּמִכְשֹׁ֤ול עֲוֹנֹו֙ יָשִׂים֙ נֹ֣כַח פָּנָ֔יו וּבָ֖א אֶל־הַנָּבִ֑יא אֲנִ֣י יְהוָ֗ה נַעֲנֵ֧יתִי לֹ֦ו בָ֖א בְּרֹ֥ב גִּלּוּלָֽיו׃ </span>|<span class="hb"><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >יִשְׂרָאֵ֡ל </span></span>|<span class="hb"><span  class="hlup" >אִ֣ישׁ </span><span  class="hlup" >מִ</span><span  class="hlup" >בֵּ֣ית </span><span  class="hlup" >יִשְׂרָאֵ֡ל </span></span>



**verse** *1*



  1.66s Done


## <: adjacent before 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

## :> adjacent after
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

In [33]:
query = '''
verse
    clause
        phrase
        <: phrase
'''
results = A.search(query)
A.table(results, start=1, end=3)

  1.66s 162555 results


n | p | verse | clause | phrase | phrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֑ים </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֑ים </span></span>|<span class="hb"><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>

In [34]:
query = '''
verse
    clause
        phrase
        :> phrase
'''
results = A.search(query)
A.table(results, start=1, end=3)

  1.66s 162555 results


n | p | verse | clause | phrase | phrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֑ים </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֑ים </span></span>

Playing with common and rare:

In [35]:
query = '''
verse
    clause
        common otype=phrase
        <: rare otype=phrase
'''
resultsCommon = A.search(query, sets=customSets)
A.table(resultsCommon, start=1, end=3)

  2.10s 25894 results


n | p | verse | clause | phrase | phrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span>וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >הָיְתָ֥ה </span></span>|<span class="hb"><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור <span  class="hlup" >כִּי־</span><span  class="hlup" >טֹ֑וב </span>וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb"><span  class="hlup" >כִּי־</span><span  class="hlup" >טֹ֑וב </span></span>|<span class="hb"><span  class="hlup" >כִּי־</span></span>|<span class="hb"><span  class="hlup" >טֹ֑וב </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב <span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֣ל </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֣ל </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span></span>|<span class="hb"><span  class="hlup" >יַּבְדֵּ֣ל </span></span>

In [36]:
query = '''
verse
    clause
        common otype=phrase
        :> rare otype=phrase
'''
resultsCommon = A.search(query, sets=customSets)
A.table(resultsCommon, start=1, end=3)

  1.88s 9671 results


n | p | verse | clause | phrase | phrase
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >Genesis 1:1</span></a><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֑ים </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2">Genesis 1:2</a>וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום <span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >מְרַחֶ֖פֶת </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב <span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֣ל </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֣ל </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >בֵּ֥ין </span><span  class="hlup" >הָ</span><span  class="hlup" >אֹ֖ור </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֥ין </span><span  class="hlup" >הַ</span><span  class="hlup" >חֹֽשֶׁךְ׃ </span></span>|<span class="hb"><span  class="hlup" >אֱלֹהִ֔ים </span></span>|<span class="hb"><span  class="hlup" >יַּבְדֵּ֣ל </span></span>

Another example: are there clauses with multiple clause atoms without a gap between the two?

In [37]:
query = '''
verse
    clause
        clause_atom
        <: clause_atom
'''
results = A.search(query)
A.table(results, start=1, end=10)

  0.63s 0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [38]:
query = '''
verse
    clause
        clause_atom
        << clause_atom
'''
results = A.search(query)
A.table(results, start=1, end=10)
A.show(results, start=1, end=1)

  0.61s 2589 results


n | p | verse | clause | clause_atom | clause_atom
--- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:7">Genesis 1:7</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">Genesis 1:7</a>וַיַּ֣עַשׂ אֱלֹהִים֮ אֶת־הָרָקִיעַ֒ <span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֗ל </span><span  class="hlup" >בֵּ֤ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֨יִם֙ </span>אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ <span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֔יִם </span>אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֗ל </span><span  class="hlup" >בֵּ֤ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֨יִם֙ </span><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֔יִם </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יַּבְדֵּ֗ל </span><span  class="hlup" >בֵּ֤ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֨יִם֙ </span></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >בֵ֣ין </span><span  class="hlup" >הַ</span><span  class="hlup" >מַּ֔יִם </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span>מַזְרִ֣יעַ זֶ֔רַע <span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span>עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span></span>|<span class="hb"><span  class="hlup" >תַּֽדְשֵׁ֤א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֨רֶץ֙ </span><span  class="hlup" >דֶּ֔שֶׁא </span><span  class="hlup" >עֵ֚שֶׂב </span></span>|<span class="hb"><span  class="hlup" >עֵ֣ץ </span><span  class="hlup" >פְּרִ֞י </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:11">Genesis 1:11</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11" sec="Genesis 1:11">Genesis 1:11</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י <span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span>אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו <span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>|<span class="hb"><span  class="hlup" >עֹ֤שֶׂה </span><span  class="hlup" >פְּרִי֙ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֹ֔ו </span></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֑רֶץ </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span>מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ <span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span>עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >תֹּוצֵ֨א </span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֜רֶץ </span><span  class="hlup" >דֶּ֠שֶׁא </span><span  class="hlup" >עֵ֣שֶׂב </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >עֵ֧ץ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:12">Genesis 1:12</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12" sec="Genesis 1:12">Genesis 1:12</a>וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ <span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span>אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו <span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span>וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>|<span class="hb"><span  class="hlup" >עֹ֥שֶׂה </span><span  class="hlup" >פְּרִ֛י </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >מִינֵ֑הוּ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:21">Genesis 1:21</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:21" sec="Genesis 1:21">Genesis 1:21</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּבְרָ֣א </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >תַּנִּינִ֖ם </span><span  class="hlup" >הַ</span><span  class="hlup" >גְּדֹלִ֑ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֣ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >נֶ֣פֶשׁ </span>הַֽחַיָּ֣ה׀ הָֽרֹמֶ֡שֶׂת אֲשֶׁר֩ שָׁרְצ֨וּ הַמַּ֜יִם <span  class="hlup" >לְ</span><span  class="hlup" >מִֽינֵהֶ֗ם </span>וְאֵ֨ת כָּל־עֹ֤וף כָּנָף֙ לְמִינֵ֔הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּבְרָ֣א </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >תַּנִּינִ֖ם </span><span  class="hlup" >הַ</span><span  class="hlup" >גְּדֹלִ֑ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֣ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >נֶ֣פֶשׁ </span><span  class="hlup" >לְ</span><span  class="hlup" >מִֽינֵהֶ֗ם </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּבְרָ֣א </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >אֶת־</span><span  class="hlup" >הַ</span><span  class="hlup" >תַּנִּינִ֖ם </span><span  class="hlup" >הַ</span><span  class="hlup" >גְּדֹלִ֑ים </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֣ת </span><span  class="hlup" >כָּל־</span><span  class="hlup" >נֶ֣פֶשׁ </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >מִֽינֵהֶ֗ם </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:29">Genesis 1:29</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29" sec="Genesis 1:29">Genesis 1:29</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים <span  class="hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span>זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ <span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span>אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃ </span>|<span class="hb"><span  class="hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span></span>|<span class="hb"><span  class="hlup" >הִנֵּה֩ </span><span  class="hlup" >נָתַ֨תִּי </span><span  class="hlup" >לָכֶ֜ם </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >עֵ֣שֶׂב׀ </span></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >הָ</span><span  class="hlup" >עֵ֛ץ </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=30&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:30">Genesis 1:30</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=30&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:30" sec="Genesis 1:30">Genesis 1:30</a><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span>אֲשֶׁר־בֹּו֙ נֶ֣פֶשׁ חַיָּ֔ה <span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hlup" >אָכְלָ֑ה </span>וַֽיְהִי־כֵֽן׃ </span>|<span class="hb"><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hlup" >אָכְלָ֑ה </span></span>|<span class="hb"><span  class="hlup" >וּֽ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >חַיַּ֣ת </span><span  class="hlup" >הָ֠</span><span  class="hlup" >אָרֶץ </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כָל־</span><span  class="hlup" >עֹ֨וף </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֜יִם </span><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hlup" >כֹ֣ל׀ </span><span  class="hlup" >רֹומֵ֣שׂ </span><span  class="hlup" >עַל־</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span></span>|<span class="hb"><span  class="hlup" >אֶת־</span><span  class="hlup" >כָּל־</span><span  class="hlup" >יֶ֥רֶק </span><span  class="hlup" >עֵ֖שֶׂב </span><span  class="hlup" >לְ</span><span  class="hlup" >אָכְלָ֑ה </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:17">Genesis 2:17</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:17" sec="Genesis 2:17">Genesis 2:17</a>וּמֵעֵ֗ץ הַדַּ֨עַת֙ טֹ֣וב וָרָ֔ע לֹ֥א תֹאכַ֖ל מִמֶּ֑נּוּ <span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֛ום </span>אֲכָלְךָ֥ מִמֶּ֖נּוּ <span  class="hlup" >מֹ֥ות </span><span  class="hlup" >תָּמֽוּת׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֛ום </span><span  class="hlup" >מֹ֥ות </span><span  class="hlup" >תָּמֽוּת׃ </span></span>|<span class="hb"><span  class="hlup" >כִּ֗י </span><span  class="hlup" >בְּ</span><span  class="hlup" >יֹ֛ום </span></span>|<span class="hb"><span  class="hlup" >מֹ֥ות </span><span  class="hlup" >תָּמֽוּת׃ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 2:22">Genesis 2:22</a>|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:22" sec="Genesis 2:22">Genesis 2:22</a><span  class="hlup" >וַ</span><span  class="hlup" >יִּבֶן֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֧ים׀ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >הַ</span><span  class="hlup" >צֵּלָ֛ע </span>אֲשֶׁר־לָקַ֥ח מִן־הָֽאָדָ֖ם <span  class="hlup" >לְ</span><span  class="hlup" >אִשָּׁ֑ה </span>וַיְבִאֶ֖הָ אֶל־הָֽאָדָֽם׃ </span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּבֶן֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֧ים׀ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >הַ</span><span  class="hlup" >צֵּלָ֛ע </span><span  class="hlup" >לְ</span><span  class="hlup" >אִשָּׁ֑ה </span></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יִּבֶן֩ </span><span  class="hlup" >יְהוָ֨ה </span><span  class="hlup" >אֱלֹהִ֧ים׀ </span><span  class="hlup" >אֶֽת־</span><span  class="hlup" >הַ</span><span  class="hlup" >צֵּלָ֛ע </span></span>|<span class="hb"><span  class="hlup" >לְ</span><span  class="hlup" >אִשָּׁ֑ה </span></span>



**verse** *1*



# Nearness

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

## =k: same start within k slots

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [39]:
results = A.search('''
chapter book=Genesis chapter=1
    clause
      =: phrase
''')

  0.14s 126 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [40]:
results=A.search('''
chapter book=Genesis chapter=1
    clause
      =0: phrase
''')

  0.14s 126 results


Now we add real freedom: 1 and 2

In [41]:
results = A.search('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''')

  0.14s 236 results


In [42]:
results = A.search('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''')

  0.14s 315 results


Let us see some cases:

In [43]:
A.table(results, start=1, end=10, linked=2)
A.show(results, condensed=False, start=1, end=4, colorMap={2: 'yellow', 3: 'cyan'})

n | p | chapter | clause | phrase
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></a></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></a></span>|<span class="hb"><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></a></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></a></span>|<span class="hb"><span  class="hlup" >חֹ֖שֶׁךְ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></a></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></a></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2" sec="Genesis 1:2"><span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></a></span>|<span class="hb"><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:3">Genesis 1:3</a>|<span class="trb">Genesis 1</span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3"><span  class="hlup" >וַ</span><span  class="hlup" >יֹּ֥אמֶר </span><span  class="hlup" >אֱלֹהִ֖ים </span></a></span>|<span class="hb"><span  class="hlup" >וַ</span></span>



**chapter** *1*





**chapter** *2*





**chapter** *3*





**chapter** *4*



The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [44]:
results = A.search('''
chapter book=Genesis chapter=1
    c:clause
    p:phrase
    
    c =2: p
''')

  0.31s 485 results


We have more results now. Here is a closer look:

In [45]:
A.table(results, start=1, end=10)
A.show(results, condensed=False, start=12, end=14, colorMap={2: 'yellow', 3: 'cyan'})

n | p | chapter | clause | phrase
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בָּרָ֣א </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span><span  class="hlup" >הָיְתָ֥ה </span><span  class="hlup" >תֹ֨הוּ֙ </span><span  class="hlup" >וָ</span><span  class="hlup" >בֹ֔הוּ </span></span>|<span class="hb"><span  class="hlup" >הָ</span><span  class="hlup" >אָ֗רֶץ </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>|<span class="hb"><span  class="hlup" >חֹ֖שֶׁךְ </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >חֹ֖שֶׁךְ </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>|<span class="hb"><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֣י </span><span  class="hlup" >תְהֹ֑ום </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >וְ</span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:2">Genesis 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span><span  class="hlup" >מְרַחֶ֖פֶת </span><span  class="hlup" >עַל־</span><span  class="hlup" >פְּנֵ֥י </span><span  class="hlup" >הַ</span><span  class="hlup" >מָּֽיִם׃ </span></span>|<span class="hb"><span  class="hlup" >ר֣וּחַ </span><span  class="hlup" >אֱלֹהִ֔ים </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:3">Genesis 1:3</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1" sec="Genesis 1">Genesis 1</a></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יֹּ֥אמֶר </span><span  class="hlup" >אֱלֹהִ֖ים </span></span>|<span class="hb"><span  class="hlup" >וַ</span></span>



**chapter** *12*





**chapter** *13*





**chapter** *14*



Here you see in results 13 and 14 a phrase of the previous clause.

Lets also play with common and rare:

In [46]:
resultsCommon = A.search('''
verse
    clause
      =: rare otype=phrase
''', sets=customSets)

  0.89s 10294 results


In [47]:
resultsCommon = A.search('''
verse
    clause
      =0: rare otype=phrase
''', sets=customSets)

  1.13s 10294 results


In [48]:
resultsCommon = A.search('''
verse
    clause
      =1: rare otype=phrase
''', sets=customSets)

  1.39s 26074 results


In [49]:
resultsCommon = A.search('''
verse
    clause
      =2: rare otype=phrase
''', sets=customSets)

  1.69s 39312 results


## :k= same end within k slots

## :k: same start and end within k slots

# Next

You are comfortable in space now.

Ready to enter a whole new dimension?
[quantifiers](searchQuantifiers.ipynb)

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
[sets](searchSets.ipynb)
relations
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
[gaps](searchGaps.ipynb)